# <div style="padding:2rem;font-size:100%;text-align:left;display:fill;border-radius:0.25rem;overflow:hidden;background-image: url(https://images.pexels.com/photos/2860804/pexels-photo-2860804.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1)"><b><span style='color:white'> PARKING ANALYSIS PREDICTOR

<img src='./images/carpark_image.jpg' alt='Carpark Image'/>

This is a collaborative group project done at the end of Phase 4 of Moringa School's Data Science program. The team members of this group include:
1. [Ezra Kipchirchir](https://github.com/dev-ezzy)
2. [Grace Mutuku](https://github.com/GraceKoki)
3. [Joy Ogutu](https://github.com/Ogutu01)
4. [Mary Gaceri](https://github.com/MaryGaceri)
5. [Mwiti Mwongo](https://github.com/M13Mwongo)

***

##  Introduction

Traffic is a nightmare, am I right? You can’t drive anywhere without being stuck in traffic for a while, especially in Nairobi. What makes it worse is that a lot of times during high-traffic periods, such as the mornings and evenings, there is a high likelihood of missing out on your desired parking spot that is near your office, especially when looking at county-run parking.
This application hopes to predict the parking patterns and likelihood of having available parking spots in certain areas at a given time of the day.

Using data analytics and machine learning techniques, we explore the field of **parking prediction** and **urban mobility**. Our research develops a state-of-the-art algorithm that can effectively estimate parking spot availability in metropolitan locations by utilizing real-time parking occupancy data from several sources inclusive of historical records. We welcome you to journey with us as we explore the inner workings of our prediction model, emphasize significant discoveries, and demonstrate how it may revolutionize urban parking system optimization.


##  Project Overview

### __Table of contents__

- [Business Understanding](#PROJECT-OVERVIEW)
- [Data Sourcing](#DATA-SOURCING)
- [Data Understanding](#DATA-UNDERSTANDING)
- [Data Preprocessing](#DATA-PREPROCESSING)
- [Exploratory Data Analysis](#EXPLORATORY-DATA-ANALYSIS)
- [Time Series Modelling](#TIME-SERIES-MODELLING)
- [Conclusion and Recommendation](#Conclusion-and-Recommendation)

### **Business Understanding**

Finding a parking place in the busy urban environments of major cities throughout the world is a problem that worries locals, commuters, and tourists equally. The need for effective parking solutions is greater than ever due to the fast urbanization, rising traffic, and expanding population. Our initiative is to change parking management in metropolitan areas confronting comparable difficulties throughout the globe by utilizing data-driven insights in response to this urgent issue. It is impossible to exaggerate the significance of this endeavor for metropolitan areas. Urban centers are the epicenters of activity, drawing millions of people for business, pleasure, and employment because they are social, cultural, and economic magnets. Nonetheless, these cities' disorganized traffic and inadequate parking facilities provide serious difficulties for local government, companies, and citizens. Our initiative intends to improve urban mobility by reducing travel times, relieving traffic congestion, and offering accurate estimates of parking spot availability.

One of the key challenges lies in accessing reliable data on parking occupancy and usage patterns. Parking spaces in urban areas are often managed by various entities, including public agencies, private operators, and informal attendants, making data collection a complex and fragmented process. Moreover, concerns about data privacy and security have hindered efforts to gather comprehensive parking data, as authorities are cautious about disclosing sensitive information due to security reasons.

Despite these challenges, our project aims to collaborate with relevant stakeholders, including:

- **City authorities**: Vital for regulatory support, infrastructure planning, and policy implementation to enhance urban mobility and parking management.
- **Parking operators**: Key players responsible for managing parking facilities, providing valuable data, and implementing innovative solutions to optimize parking spot utilization.
- **Technology partners**: Essential for developing and implementing data-driven tools, such as predictive models and smart parking systems, to improve parking availability and streamline operations.
- **Motorists (end-users)**: The primary beneficiaries of improved parking management solutions, as they will benefit from reduced search time, enhanced convenience, and better access to parking spots.

By fostering partnerships and promoting transparency, we seek to establish a data-sharing framework that respects privacy concerns while enabling the development of innovative solutions to improve parking management in urban centers worldwide. Optimizing parking and reducing congestion, enhances business efficiency, attracts investments, and stimulates economic activity. Encouraging alternative transportation modes reduces emissions and contributes to environmental conservation. Through data analytics, stakeholder collaboration, and innovative tech, we aim to create smarter, more efficient urban mobility ecosystems benefiting all.

#### **Problem Statement**

The absence of accurate and up-to-date data on parking spot availability not only impedes the development of effective predictive models but also limits the implementation of innovative solutions aimed at addressing urban mobility challenges. Without access to comprehensive data sources, parking prediction systems struggle to provide reliable real-time information, leading to suboptimal parking decisions and increased traffic congestion. Overcoming these challenges is crucial for creating a parking prediction system that not only improves parking navigation but also contributes to the overall sustainability and livability of urban areas by enhancing economic productivity, and fostering a more seamless urban mobility experience for all stakeholders.

#### **Objectives**

##### **MAIN OBJECTIVE**
Develop a robust time series-based parking spot predictor that accurately forecasts parking spot availability in urban areas, leveraging historical parking data and real-time variables.

##### **OTHER OBJECTIVES**
1. To collect and preprocess historical parking data from various sources and integrate relevant time-varying features, such as time of day, day of the week and holidays into the predictive model.

2. To explore various time series forecasting techniques, including ARIMA and SARIMA and evaluate the performance of each technique using metrics like accuracy, precision, recall, and F1-score. 

3. To develop and deploy a user-friendly interface or mobile application that allows motorists to access real-time parking predictions and navigate to available parking spots efficiently.

### **Important Background Information**

#### **a) Imports & OOP**

The necessary libraries were first imported.

In [1]:
# Importing necessary libraries
# Basics
import pandas as pd
import numpy as np
import math
import itertools
import requests
import json
from io import StringIO
from datetime import datetime, timedelta
from requests import api
import time

# Visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline 
import plotly.express as px
import seaborn as sns
import matplotlib.patches as mpatches
from matplotlib.pylab import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose

# Modeling libraries
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split        
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, adfuller
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline 
from pmdarima import auto_arima      
from prophet import Prophet 
from sklearn.model_selection import GridSearchCV

#Model deployment libraries
# import streamlit 


# Warnings
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.filterwarnings('ignore')

# Custom Options for displaying rows.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',100)

In object-oriented programming (OOP), classes serve as blueprints, dividing code into modular components that contain data and actions. They encourage encapsulation, abstraction, and inheritance in code, making it more modular, readable, and manageable. Classes offer an organized way to developing and implementing code, promoting clarity and efficiency in software development.

Consequently, the following classes were implemented and defined below:
1. Data Sourcing
2. Data Understanding
3. Data Preprocessing
4. Data Analysis
5. Modelling

In [2]:
class DataSourcing:
  def __init__(self,df_carparks,df_carpark_structure,df_carpark_history,df_carparks_zones_merged,df_carparks_zones_coords_merged,df_holidays):
    
    self.carparks_all = df_carparks
    self.carpark_structure = df_carpark_structure
    self.carpark_history = df_carpark_history
    self.carparks_zones = df_carparks_zones_merged
    self.carparks_zones_coords = df_carparks_zones_coords_merged
    self.holidays = df_holidays

In [3]:
class DataUnderstanding(DataSourcing):
  def __init__(self, data_sourcing_object):
    if (isinstance(data_sourcing_object, DataSourcing)):
      self.carparks_all = data_sourcing_object.carparks_all
      self.carpark_structure = data_sourcing_object.carpark_structure
      self.carpark_history = data_sourcing_object.carpark_history
      self.holidays = data_sourcing_object.holidays
    else:
      raise TypeError('data_sourcing_object must be an instance of DataSourcing')

  def carpark_names(self):
    return self.carparks_all
  
  def carpark_details(self):
    message = f"""
    There are {self.carparks_all.shape[0]} carparks in the dataset.
    
    The highest number of parking spots available is {self.carpark_structure['spots'].max()}, found at {self.carpark_structure.loc[self.carpark_structure['spots'].idxmax(), 'facility_name']}.
    
    The lowest number of parking spots available is {self.carpark_structure['spots'].min()}, found at {self.carpark_structure.loc[self.carpark_structure['spots'].idxmin(), 'facility_name']}.
    
    There are {len(self.carpark_structure.columns)} columns in the dataset: namely {self.carpark_structure.columns.to_list()}
    """
    print(message)
    return None

  def examine_carpark_history(self):
    print(" ################### Details about the data ################### \n ")
    print(f"The dataset is a DataFrame with {self.carpark_history.shape[0]} rows and {self.carpark_history.shape[1]} columns\n")
    print("Columns of the dataset:", self.carpark_history.columns.to_list())
    print("\nFirst 5 records of the dataset ")
    display(self.carpark_history.head())
    
    # Display information about the dataset
    print("\nData information")
    display(self.carpark_history.info())
    print("\nNull Values ")
    display(self.carpark_history.isnull().sum())
    # print("\nDuplicate Values ")
    # print(self.carpark_history.duplicated(), 'duplicate values')
    display(self.carpark_history.describe())
  
    print('\nData Details')
    print(f'Number of unique Parking Facilities:', self.carpark_history.facility_name.nunique())
    # print(f'Number of unique days:', self.carpark_history.date.nunique())
    
    return None


In [4]:
class DataPreprocessing(DataUnderstanding):
  def __init__(self, data_understanding_object):
    if (isinstance(data_understanding_object, DataUnderstanding)):
      self.carparks_all = data_understanding_object.carparks_all
      self.carpark_structure = data_understanding_object.carpark_structure
      self.carpark_history = data_understanding_object.carpark_history
      self.holidays = data_understanding_object.holidays
    else:
      raise TypeError('data_understanding_object must be an instance of DataUnderstanding')

  def drop_duplicate_carpark_history(self):
    self.carpark_history = self.carpark_history.drop_duplicates()
    return self.carpark_history

  def drop_facility_ids(self):
    # Drop records where facility_id is between 486 and 490
    self.carpark_structure = self.carpark_structure[~ self.carpark_structure['facility_id'].between(486, 490)]

    # Drop records where facility_id is between 1 and 5
    self.carpark_structure = self.carpark_structure[~ self.carpark_structure['facility_id'].between(1, 5)]
    
    return self.carpark_structure
  
  def merge_zones_and_carpark_history(self):
    # TODO - TEST THIS FUNCTION TO MAKE SURE IT WORKS
    # Creating merged dataframe
    df = pd.merge(self.carpark_history, self.carpark_history_zones_only, how='outer',left_index=True, right_index=True)
    
    # Dropping the zones column now that the data is merged
    df.drop(columns=['zones'],inplace=True)
    
    # Renaming the spots column to total_parking_spots
    df.rename(columns={'spots':'total_parking_spots'},inplace=True)
    
    # Ensuring the occupancy_total and total_parking_spots are integers
    df['occupancy_total'] = df['occupancy_total'].astype(int)
    df['total_parking_spots'] = df['total_parking_spots'].astype(int)
    
    # Assigning the df to self.carpark_history
    self.carpark_history = df

    return self.carpark_history

  def merge_coords_and_carpark_history(self):
    # Creating merged dataframe
    merged_df = self.carpark_history.merge(self.facility_coordinates, on='facility_id',how='left')
    
    # Update 'longitude' and 'latitude' columns where the condition is met
    merged_df['longitude'].combine
    
    self.facility_coordinates['longitude'] = merged_df['longitude'].combine_first(self.facility_coordinates['longitude'])
    self.facility_coordinates['latitude'] = merged_df['latitude'].combine_first(self.facility_coordinates['latitude'])
    
    self.carpark_history = merged_df
    
    return self.carpark_history
  
  def save_dataframe_to_parquet(self,dataframe,path):
    dataframe.to_parquet(path)
    print("File saved!")
    return None

  def extract_date_time_dayOfWeek(self):
    # Extracting date and time from MessageDate
    self.carpark_history[['date','time']] = self.carpark_history['message_date'].str.split('T',expand=True)
    
    # Using the newly created fields to establish the day of the week
    self.carpark_history['day_of_Week'] = pd.to_datetime(self.carpark_history['date']).dt.day_name()
    
    # Converting time to timedelta object
    self.carpark_history['time'] = pd.to_datetime(self.carpark_history['time']).dt.time
    
    # Extracting hour from time
    self.carpark_history['hour'] = pd.to_datetime(self.carpark_history['time']).dt.hour
    
    # Creating time_category column
    self.carpark_history['time_category'] = self.carpark_history['hour'].apply(lambda x: self.categorize_time(x))
    
    # Convert date to datetime object
    self.carpark_history['date'] = pd.to_datetime(self.carpark_history['date'])
    
    # Extract month name from date and assign it to a new month column
    self.carpark_history['month'] = self.carpark_history['date'].dt.strftime('%B')
    
    # Convert time to time object
    self.carpark_history['time'] = pd.to_datetime(self.carpark_history['time']).dt.time
    
    # Drop unnecessary columns
    self.carpark_history.drop(['MessageDate', 'hour'], axis=1, inplace=True) 
    
    return self.carpark_history.head()

  def categorize_carpark_time(self):
    self.carpark_history['time_category'] = self.carpark_history['time'].apply(lambda x: self.categorize_time(x.hour))
    
    return self.carpark_history

  def drop_columns(self,columns_to_keep):
    # Drop columns in zones
    self.carpark_history = self.carpark_history[columns_to_keep]
    return self.carpark_history

  def rename_columns(self, column_names):
    self.carpark_history = self.carpark_history.rename(columns=column_names)
    return self.carpark_history
  
  def create_parking_availability(self):
    self.carpark_history['parking_availability'] = self.carpark_history['capacity'] - self.carpark_history['occupancy']
    return self.carpark_history
  
  def holiday_formatting(self):
    # Convert 'date' column to datetime format
    self.holidays['date'] = pd.to_datetime(self.holidays['date'], format='%b %d')

    # Extract month and day information and format it as 'Month Day' in self.holidays
    self.holidays['month_day'] = self.holidays['date'].dt.strftime('%m-%d')

    # Create a set of holiday month-day combinations
    holidays_month_day = set(self.holidays['month_day'])

    # Check if the month-day combination of each date in df1 matches any holiday month-day combination
    self.carpark_history['month_day'] = self.carpark_history['date'].dt.strftime('%m-%d')
    self.carpark_history['is_holiday'] = self.carpark_history['month_day'].isin(holidays_month_day)

    # Map True/False to 'yes'/'no' for 'is_holiday' column
    self.carpark_history['is_holiday'] = self.carpark_history['is_holiday'].map({True: 'Yes', False: 'No'})

    # Drop the temporary 'month_day' column
    self.carpark_history.drop(columns=['month_day'], inplace=True)

    # Display the DataFrame to verify changes
    return self.carpark_history.head()
  
  
  # Helper functions that do not directly modify content in the object instance  
  def categorize_time(hour):
    if 4 <= hour < 7:
        return 'Early Morning'
    elif 7 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 15:
        return 'Afternoon'
    elif 15 <= hour < 18:
        return 'Late Afternoon'
    elif 18 <= hour < 20:
        return 'Evening'
    else:
        return 'Night'

In [5]:
# WARN - Potential issue with how the data analysis constructor has been made. Test it out and find out
class DataAnalysis(DataPreprocessing):
  def __init__(self, data_preprocessing_object):
    super().__init__(data_preprocessing_object)

  def preprocess_dataframe(self):
    # Group by 'date' and 'facility_name', and calculate the mean for 'parking_availability' and 'occupancy'
    processed_df = self.carpark_history.groupby(['date', 'facility_name']).agg({'parking_availability': 'mean', 'occupancy': 'mean', 'capacity': 'mean', 'longitude' : 'mean', 'latitude' : 'mean'}).reset_index()

    return processed_df
  
  def create_distribution_plots(self):
    # List of columns to include in distribution plots
    columns = ['capacity', 'parking_availability', 'occupancy', 'longitude', 'latitude']

    # Preprocess the dataframe to calculate means for 'parking_availability' and 'occupancy'
    processed_df = self.preprocess_dataframe(self.carpark_history)

    # Loop through each column in the DataFrame
    for column in columns:
        # Check if the column is numerical
        if pd.api.types.is_numeric_dtype(processed_df[column]):
            # Create a figure with subplots
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 5))

            # Histogram with KDE
            sns.histplot(processed_df[column], kde=True, ax=axes[0])
            axes[0].set_title(f'Distribution Plot for {column}')
            axes[0].set_xlabel(column)
            axes[0].set_ylabel('Frequency')

            # Boxplot
            sns.boxplot(x=processed_df[column], ax=axes[1])
            axes[1].set_title(f'Boxplot for {column}')
            axes[1].set_xlabel(column)
            axes[1].set_ylabel('')

            plt.tight_layout()
            plt.show()

  

In [6]:
class Modelling(DataAnalysis):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)


The API - whose base URL was `https://api.transport.nsw.gov.au/v1/carpark` - had two endpoints:
1. `{baseURL}?facility={facility_id}` - Containts one optional variable ***facility_id***. Returns occupancy details of a car park based on a facility ID. If the facility ID specified, a list of facility names with their ID will be returned.
2. `{baseURL}}/history?facility={facility_id}&eventdate={date_in_question}` - Contains two mandatory variables, ***facility_id*** and ***date_in_question*** formatted as *YYYY-MM-DD*. Returns historical occupancy details of a car park based on a facility ID
and event date. 

#### **b) The Process of Fetching Data**

Our data was sourced from the Transport for New South Wales(TfNSW) website, more speficially, from their [Car Park API](https://opendata.transport.nsw.gov.au/dataset/car-park-api).

Our intention was to use this API to fetch six months' worth of historical parking data. An extensive time period would lead to a proper understanding of parking habits across a wide array of conditions while factoring in social events, public holidays, school holidays and even leave days of employees.

The team came up with code to automatically make requests to the API, and save this information in a dataframe. However, after further study of the API's structure and the data being received, the team saw it best to have these requests made once and the resulting data stored in json files, which can be read by pandas.

The function below was used to retrieve car park data from the TfNSW API and saves it to a JSON file. It will then read the JSON file into a dataframe, rename the columns as they come with no name from the API

```python
def get_carparks_list():
  dotenv.load_dotenv('.env')
  # path to json file created/saved
  carparks_file_path = './data/carparks.json'
  # Delete any existing file at carparks path
  os.remove(carparks_file_path) if os.path.exists(carparks_file_path) else None

  # Creating header for request
  headers = {
      "Authorization": f"apikey {os.environ.get('apikey')}"
  }
  # Specifying url to get carparks
  url_carparks = 'https://api.transport.nsw.gov.au/v1/carpark'

  list_of_carparks = requests.get(url_carparks, headers=headers).json()

  df_carparks = pd.DataFrame.from_dict(list_of_carparks, orient='index')
  # Resetting the index to label the columns afterwards
  df_carparks = df_carparks.reset_index()
  df_carparks.columns = ['facility_id', 'CarParkName']

  # Deleting old file
  os.remove(carparks_file_path) if os.path.exists(carparks_file_path) else None

  # Creating new file with updated column titles
  pd.DataFrame.to_json(df_carparks, carparks_file_path)

  print('File created and updated successfully.')
  return
```

Having the names of the various facilites, the structure of each of the carparks was investigated. It was noted that each car park can have a different configuration, where each facility may have one or more car parks, and each car park may have one or more zones as depicted below.

<div style="text-align:center">
<img src='./images/carpark_structure.png' alt='Carpark structure'>
</div>

Knowing this, the function below was created to fetch the individual details of the carparks - using the JSON file just created - to properly scrutinise their structure. This would then be saved in its own JSON file named `carpark_structure.json` for future reference.

```python
def get_carpark_structure(path_to_carpark_json_file):
  # Delete file found at same path
  os.remove('./data/carpark_structure.json') if os.path.exists('./data/carpark_structure.json') else None

  # Add file to dataframe
  df_carparks = pd.read_json(path_to_carpark_json_file)
  # Initialise array that will hold information
  carpark_details_array = []

  # Loop through carparks to get information
  for index, row in df_carparks.iterrows():
    facility = row['facility_id']
    url = f'https://api.transport.nsw.gov.au/v1/carpark?facility={facility}'

    # Creating header for request
    headers = {
        "Authorization": f"apikey {os.environ.get('apikey')}"
    }
    # Make request
    response = requests.get(url, headers=headers).json()

    # Add to array
    carpark_details_array.append(response)

  # Store information in JSON file
  with open('./data/carpark_structure.json', 'w') as f:
    json.dump(carpark_details_array, f)
  # Create dataframe and return it
  return pd.DataFrame(carpark_details_array)
```

Having done that, a new function - named `date_getter` - was created to give a list of all the days in a given time period. This function generates a list of dates based in the input time delta based, taking a time delta as an argument and returns a list of dates in the format "YYY-MM-DD".

This would be useful as carpark history for each of the carparks within a given time delta would be needed.

```python
def date_getter(td):
    # Array that stores the dates to be searched for
    date_period_list = []

    # The last date to be searched for
    cutoff_date = datetime(2023, 12, 31)
    target_date = cutoff_date - td

    # Ensure that records of each day are obtained
    delta = timedelta(days=1)

    while target_date <= cutoff_date:
        date_period_list.append(target_date.strftime("%Y-%m-%d"))
        target_date += delta

    return date_period_list
```

Having a date function, a new function (`get_carpark_history`) was made to fetch the carpark history of a particular facility across a range of dates.

This function is used to get carpark history data for a specific facility and dates, taking the name of the carpark facility and the list of dates for which to retrieve carpark history data as arguments. It returns a dataFrame containing the carpark history data, while saving the data into a file.

```python
def get_carpark_history(facility, dates_array):

    # Initialize data array
    data_array = []

    # Define the path for the JSON file
    json_file_path = f"./data/carpark history/facility_{facility}.json"

    # Set the request header
    headers = {
        "Authorization": f"apikey {os.environ.get('apikey')}"
    }

    # Delete the file if it exists
    if os.path.exists(json_file_path):
        os.remove(json_file_path)

    # Make a request for each date and aggregate the data
    for date in dates_array:
        url = f'https://api.transport.nsw.gov.au/v1/carpark/history?facility={facility}&eventdate={date}'
        response = requests.get(url, headers=headers).json()

        if data_array == []:
            data_array = response
        else:
            data_array = data_array + response

    # Save the data to a JSON file
    with open(json_file_path, 'w') as f:
        json.dump(data_array, f)

    # Read the JSON file
    with open(json_file_path) as f:
        data = json.load(f)

    # Convert the read data into a pandas DataFrame
    return pd.DataFrame(data)
```

### **1. Data Sourcing:**

Our data was sourced from the Transport for New South Wales(TfNSW) website, more speficially, from their [Car Park API](https://opendata.transport.nsw.gov.au/dataset/car-park-api).

The API - whose base URL was `https://api.transport.nsw.gov.au/v1/carpark` - had two endpoints:
1. `{baseURL}?facility={facility_id}` - Containts one optional variable ***facility_id***. Returns occupancy details of a car park based on a facility ID. If the facility ID specified, a list of facility names with their ID will be returned.
2. `{baseURL}}/history?facility={facility_id}&eventdate={date_in_question}` - Contains two mandatory variables, ***facility_id*** and ***date_in_question*** formatted as *YYYY-MM-DD*. Returns historical occupancy details of a car park based on a facility ID
and event date. 

Data was sourced over a 6 month period, from the beginning of July 2023 to 31st December 2023. A loop was created for each facility using the given date range, and the `get_carpark_history` function was run within that loop. The respective files that were saved contained the parking history of that facility for the 6-month time period (found in *./data/carpark_history_6_months/facility_<<facility_id>>*). However, in a bid to simplify the starting point and to ensure that one dataframe is used as our starting point, the code below was implemented to read all the data from the various parquet files and put it in one file, from which the one dataframe was created.

```python
df = pd.DataFrame()

for file in os.listdir('data/carpark_history_6_months'):
  df_file = pd.read_parquet('data/carpark_history_6_months/' + file)
    
    if file == 'facility_6.parquet':
      df = df_file
    else:
      df = pd.concat([df,df_file]).reset_index(drop=True)

# Save to parquet
df.to_parquet('data/carpark_history_6_months.parquet')
```

The parquet file was chosen due as its columnar storage format is highly efficient for both reading and writing large datasets due to its compression and columnar layout.

In [7]:
df_carpark_history = pd.read_parquet('./data/carpark_history_6_months.parquet')


Moving on, the data containing the parking lot structure as well as the parking lot names can now be converted to a dataframe. 

In [8]:
df_carparks = pd.read_json('./data/carparks.json')
df_carpark_structure = pd.read_json('./data/carpark_structure.json')

Furthermore, the files containing the geolocation coordinates of each parking facility, as well as the public holiday information for Australia were loaded into their own respective dataframes. This information will come in handy later on. Files with the carparks and zones merged, as well as the carparks, zones and coordinates merged, were loaded into their own dataframes.

In [9]:
# Loading the holiday data
df_holidays = pd.read_csv('./data/NSW_holidays_2023.csv')

In [10]:
# Creating the dataframe with carparks and zones merged
df_carparks_zones_merged = pd.read_parquet('./data/carpark_history_6_months_with_zones.parquet')

# Dataframe for the carparks, zones and coordinates merged
df_carparks_zones_coords_merged = pd.read_parquet('./data/carpark_history_6_months_zones_coords.parquet')

Having done this, all the dataframes can now be passed onto the DataSourcing class

In [11]:
data_sourcing = DataSourcing(df_carparks,df_carpark_structure,df_carpark_history,df_carparks_zones_merged,df_carparks_zones_coords_merged,df_holidays)

### **2. Data Understanding:**

The identification, gathering, and cursory analysis of the data in this part will be carried out by:

- Gathering preliminary data, which has been put into a JSON file.
- Describing the data that we have at our disposal.
- Looking for patterns and correlations in the data.
- Confirming the accuracy of the data.

Instantiating the data understanding class

In [12]:
data_understanding = DataUnderstanding(data_sourcing)

Having done this, a general summary of the all the carparks' parking history is outlined:

In [13]:
data_understanding.examine_carpark_history()

 ################### Details about the data ################### 
 
The dataset is a DataFrame with 2924545 rows and 10 columns

Columns of the dataset: ['tsn', 'time', 'spots', 'zones', 'ParkID', 'occupancy', 'MessageDate', 'facility_id', 'facility_name', 'tfnsw_facility_id']

First 5 records of the dataset 


,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id
0,207210,741448862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:01:02,6,Gordon Henry St North Car Park,207210TPR001
1,207210,741450092,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:21:32,6,Gordon Henry St North Car Park,207210TPR001
2,207210,741468991,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:36:31,6,Gordon Henry St North Car Park,207210TPR001
3,207210,741469862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:51:02,6,Gordon Henry St North Car Park,207210TPR001
4,207210,741474196,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T07:03:16,6,Gordon Henry St North Car Park,207210TPR001



Data information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924545 entries, 0 to 2924544
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   tsn                int32 
 1   time               int32 
 2   spots              int32 
 3   zones              object
 4   ParkID             int32 
 5   occupancy          object
 6   MessageDate        object
 7   facility_id        int32 
 8   facility_name      object
 9   tfnsw_facility_id  object
dtypes: int32(5), object(5)
memory usage: 167.3+ MB


None


Null Values 


tsn                  0
time                 0
spots                0
zones                0
ParkID               0
occupancy            0
MessageDate          0
facility_id          0
facility_name        0
tfnsw_facility_id    0
dtype: int64

,tsn,time,spots,ParkID,facility_id
count,2.924545e+06,2.924545e+06,2.924545e+06,2924545.0,2.924545e+06
mean,8.738508e+05,7.501181e+08,7.663994e+02,1.0,2.013059e+01
std,8.994475e+05,4.247952e+06,5.052506e+02,0.0,7.783199e+00
min,2.072100e+05,7.414488e+08,4.200000e+01,1.0,6.000000e+00
25%,2.179330e+05,7.468320e+08,3.730000e+02,1.0,1.500000e+01
50%,2.750100e+05,7.504562e+08,7.000000e+02,1.0,2.000000e+01
75%,2.126158e+06,7.536812e+08,1.057000e+03,1.0,2.700000e+01
max,2.155384e+06,7.573462e+08,1.884000e+03,1.0,3.300000e+01



Data Details
Number of unique Parking Facilities: 28


We can further look at the names and facility_ids of the various carparks.

In [14]:
data_understanding.carparks_all

,facility_id,CarParkName
0,1,Tallawong Station Car Park (historical only)
1,10,Warriewood Car Park
2,11,Narrabeen Car Park
3,12,Mona Vale Car Park
4,13,Dee Why Car Park
5,14,West Ryde Car Park
6,15,Sutherland East Parade Car Park
7,16,Leppington Car Park
8,17,Edmondson Park South Car Park
9,18,St Marys Car Park


A closer look at the detailed structure of the carparks was also done.

In [15]:
data_understanding.carpark_structure

,tsn,time,spots,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id
0,2155384,742877319,1004,"[{'spots': '152', 'zone_id': 'CPS-CUD1', 'occu...",1,"{'loop': None, 'total': '981', 'monthlies': No...",2023-07-17T12:48:39,1,Tallawong Station Car Park,2155384CCP001
1,2101131,760433800,244,"[{'spots': '244', 'zone_id': '1', 'occupancy':...",1,"{'loop': None, 'total': '75', 'monthlies': Non...",2024-02-05T18:36:40,10,Warriewood Car Park,2101131TPR001
2,2101130,760434123,46,"[{'spots': '46', 'zone_id': '1', 'occupancy': ...",1,"{'loop': '63843', 'total': '14', 'monthlies': ...",2024-02-05T18:42:03,11,Narrabeen Car Park,2101130TPR001
3,2103108,760434091,68,"[{'spots': '68', 'zone_id': '1', 'occupancy': ...",1,"{'loop': None, 'total': '44', 'monthlies': Non...",2024-02-05T18:41:31,12,Mona Vale Car Park,2103108TPR001
4,2099207,760433747,117,"[{'spots': '117', 'zone_id': '1', 'occupancy':...",1,"{'loop': '53269', 'total': '52', 'monthlies': ...",2024-02-05T18:35:47,13,Dee Why Car Park,2099207TPR001
5,211420,760433971,151,"[{'spots': '151', 'zone_id': '1', 'occupancy':...",1,"{'loop': '228018', 'total': '31', 'monthlies':...",2024-02-05T18:39:31,14,West Ryde Car Park,211420TPR001
6,223210,760434193,373,"[{'spots': '373', 'zone_id': '1', 'occupancy':...",1,"{'loop': None, 'total': '87', 'monthlies': Non...",2024-02-05T18:43:13,15,Sutherland East Parade Car Park,223210TPR001
7,217933,760434198,1884,"[{'spots': '1884', 'zone_id': '1', 'occupancy'...",1,"{'loop': '809193', 'total': '434', 'monthlies'...",2024-02-05T18:43:18,16,Leppington Car Park,217933TPR001
8,217426,760434201,1429,"[{'spots': '1429', 'zone_id': '1', 'occupancy'...",1,"{'loop': '740938', 'total': '136', 'monthlies'...",2024-02-05T18:43:21,17,Edmondson Park South Car Park,217426TPR001
9,276010,760434199,682,"[{'spots': '682', 'zone_id': '1', 'occupancy':...",1,"{'loop': None, 'total': '142', 'monthlies': No...",2024-02-05T18:43:19,18,St Marys Car Park,276010TPR001


A summary of the carparks was shown as well

In [16]:
data_understanding.carpark_details()


    There are 38 carparks in the dataset.
    
    The highest number of parking spots available is 1884, found at Leppington Car Park.
    
    The lowest number of parking spots available is 42, found at Kiama Car Park.
    
    There are 10 columns in the dataset: namely ['tsn', 'time', 'spots', 'zones', 'ParkID', 'occupancy', 'MessageDate', 'facility_id', 'facility_name', 'tfnsw_facility_id']
    


### **3. Data Preprocessing:**

A new instance of the DataPreprocessing object is first instantiated

In [17]:
data_preprocessing = DataPreprocessing(data_understanding)

From here, various steps in the data preprocessing lifecycle can be carried out

#### **Dealing with Outliers**

Outliers were kept in the dataset for a variety of reasons. Preserving outliers ensures the integrity of the data by accurately reflecting the underlying distribution and real-world phenomena. This decision helps to prevent valuable information from being discarded, which could potentially skew subsequent analyses or models. 

Furthermore, retaining outliers guards against bias being introduced into the analysis, as extreme values may represent genuine data points rather than errors. Additionally, the presence of outliers contributes to the robustness of analysis, as it ensures that statistical methods and machine learning algorithms are not unduly influenced by extreme values. Moreover, outliers can provide insights into unique patterns or anomalies within the data, prompting further investigation. 

Lastly, analyzing outliers aids in identifying data quality issues such as measurement errors or data entry mistakes, facilitating improvements in data collection processes and overall data quality. Thus, while the decision to keep or remove outliers depends on the specific context and objectives of the analysis, retaining outliers is often essential for ensuring accurate and comprehensive data analysis.

#### **Dropping unneccesary features/duplicates & Renaming Columns**

In [19]:
data_preprocessing.carpark_history

We will start by dropping the columns that aren't needed.

In [18]:
data_preprocessing.drop_columns()

KeyError: "['spots_x', 'occupancy.total', 'longitude', 'latitude'] not in index"

We will then proceed to rename certain columns

In [ ]:
data_preprocessing.rename_columns({'spots_x': 'capacity','occupancy.total': 'occupancy'})

We will also go ahead and drop duplicate values

In [ ]:
data_preprocessing.drop_duplicate_carpark_history()

#### **Feature Engineering**

The dataframe contains a crucial column known as `zones` - which is recommended by the API to use - contains crucial information for calculating the parking availability. The column, which contains a list of dictionaries, is extracted from the zone and placed in its own dataframe using the following code:

```python
def separate_zones_from_carpark_history(self):

    # Converting the zones column to its own dataframe
    df_zones = pd.DataFrame(columns=['spots', 'zone_id', 'zone_name', 'parent_zone_id', 'occupancy.loop','occupancy.monthlies','occupancy.open_gate','occupancy.total','occupancy.transients'])
    rename_format = {
        0: 'spots',
        1: 'zone_id',
        2: 'zone_name',
        3: 'parent_zone_id',
        4: 'occupancy_loops',
        5: 'occupancy_total',
        6: 'occupancy_monthlies',
        7: 'occupancy_open_gate',
        8: 'occupancy_transients'
    }

    zones_list = []

    for index,row in self.carpark_history.iterrows():
        # Normalize values in each record in zones column
        df_zone = pd.json_normalize(row['zones'])
        
        zones_list.append(df_zone)

    # Concatendating zones list
    df_zones = pd.concat(zones_list, ignore_index=True)
    
    # Keeping necessary columns
    self.carpark_history_zones_only = df_zones[['zone_id','occupancy_total']]

    return self.carpark_history_zones_only
```

We will now proceed to create a new column `parking_availability` which is the difference between the capacity and the occupancy

In [ ]:
data_preprocessing.create_parking_availability()

Columns for the date and time were created from the MessageDate column by spliting it across a common letter. A new column that also had the time_category was created as well

In [ ]:
data_preprocessing.extract_date_time_dayOfWeek()

Infirmation on the holidays was now included in the carpark history dataframe

In [ ]:
data_preprocessing.holiday_formatting()

#### **Adding Geolocation Data**

The lat/long coordinates for each of the carparks was obtained from the API's documentation that the data was sourced and saved in the `coords.json` file. 

It was combined with the carparks history with zones to create a mew file: `carpark_history_6_months_zones_coords.parquet`

In [ ]:
data_preprocessing.merge_coords_and_carpark_history()

#### **Data Cleaning**

Certain facilities were deemed surplus to requirements when it comes to analysis and modelling. These facilities were the ones with facility_ids 1-5 and 486-490 (inclusive). 

For facility_ids 1-5, these facilities did not have data for the time period in question. As for facility_ids 486-490, these facilities - as per the API we sourced the data from - had inconsistent data. 

In [ ]:
data_preprocessing.drop_facility_ids()

### **4. Explorative Data Analysis & Visualisation**

In this section, through a combination of visualizations, statistical summaries, and data manipulation techniques, we delve into the dataset's intricacies, examining the distribution of variables, identifying correlations, and detecting anomalies. By thoroughly exploring the data's structure and characteristics, we aim to gain a deeper understanding of its underlying properties, paving the way for informed hypotheses and refined analysis strategies.

First we will start by instantiating a DataAnalysis object

In [ ]:
data_analysis = DataAnalysis(data_understanding)

#### **Univariate Analysis**

The distribution of the numerical columns is visualized below using distribution plots (histogram with KDE) and boxplots.

In [ ]:
data_analysis.carpark_history.columns

In [ ]:
# TODO - Insert code here

**Capacity**

1. **Distribution Plot for Capacity**:
   - This graph illustrates the frequency distribution of capacity. On the x-axis, we have capacity values, and on the y-axis, we see the frequency. The vertical bars represent the number of occurrences (frequency) for each capacity value or range. Additionally, a smooth line overlays the bars to show the distribution trend. Notably, most data points cluster around the 0-250 capacity range, indicating higher frequency in that interval.

2. **Boxplot for Capacity**:
   - The box represents the interquartile range where 50% of the data exists. Lines extending from either end of the box (whiskers) indicate variability beyond this range, and any points beyond these whiskers are considered to be outliers.

The majority of parking facilities have a capacity clustered around the 0-250 range, with few outliers indicating larger parking capacities. The boxplot suggests that most facilities have a relatively consistent capacity, with some variability observed.

**Parking Availability**


1. **Distribution Plot for parking availability**:
   - This graph illustrates the frequency distribution of parking availability. On the x-axis, we have different ranges of parking availability (from 0 to 1750), and on the y-axis, we see the frequency of occurrence. The blue bars represent the number of occurrences (frequency) for each parking availability value or range.
   - Notably, there's a significant peak at **zero parking availability**, indicating that this value occurs most frequently. As parking availability increases, the frequency decreases.
   - A smooth line overlaid on top of the bars indicates the frequency trend more smoothly.

2. **Boxplot for parking availability**:
   - The interquartile range (IQR) is approximately between **0 and 500** for parking availability.
   - Several **outliers** are present beyond the whiskers of the boxplot.

The distribution plot indicates a significant peak at zero parking availability, suggesting that many parking facilities frequently reach full capacity. As availability increases, the frequency of occurrence decreases. The boxplot shows that the interquartile range (IQR) for parking availability is relatively small, indicating less variability in this metric compared to capacity.


**Occupancy**


1. **Distribution Plot for occupancy**:
   - This graph illustrates the frequency distribution of occupancy. On the x-axis, we have different levels of occupancy (ranging from 0 to 800), and on the y-axis, we see the frequency of occurrence. The blue bars represent the number of occurrences (frequency) for each level of occupancy.
   - Notably, there's a significant peak at **low levels of occupancy**, indicating that these values occur most frequently. As occupancy increases, the frequency decreases.
   - A smooth line overlaid on top of the bars indicates the distribution trend more smoothly.

2. **Boxplot for occupancy**:
   - The interquartile range (IQR) is large at lower occupancies, suggesting variability in the data.
   - Several **outliers** are visible at higher occupancies above 500.

Similar to parking availability, there's a peak at low levels of occupancy, indicating frequent instances of low usage. As occupancy increases, the frequency decreases. The boxplot highlights variability in occupancy levels, with some facilities experiencing higher occupancy rates and potential outliers.

**Longitude**


1. **Distribution Plot for longitude**:
   - This graph illustrates the frequency distribution of longitude data. On the x-axis, we have different longitude values, ranging from approximately **-34.6** to **-33.4**. The y-axis represents the **frequency**, with values ranging up to **1200**.
   - Notably, there's a prominent peak around **-33.8**, indicating that this longitude value occurs most frequently.
   - A smooth line overlays the bars, indicating the distribution trend.

2. **Boxplot for longitude**:
   - The interquartile range (IQR) is represented by the blue box, which spans from approximately **-34.6** to **-33.4**.
   - Lines (whiskers) extend from the box, indicating variability beyond the upper and lower quartiles. Some individual points beyond the whiskers may be potential **outliers**.

**Latitude**


1. **Distribution Plot for Latitude**:
    - On the left side, there's a distribution plot showing the frequency of data points at different latitudes.
    - The x-axis represents "latitude" with values ranging from 150.7 to 151.3.
    - The y-axis represents "Frequency" with values ranging from 0 to 700.
    - Notably, there are peaks in frequency around latitudes 150.8 and 151.2.

2. **Boxplot for Latitude**:
    - On the right side, there's a boxplot illustrating the spread of the latitude data.
    - The x-axis labels are similar to those in the distribution plot.

The distribution plots show the frequency of data points at different longitude and latitude values. While there are peaks indicating common longitude and latitude values, the boxplots reveal variability in these geographic coordinates, with potential outliers suggesting locations that deviate from the norm.


##### **Count Plot of Non-Holidays and Holidays from July to December**

In [ ]:
# Define the order of months from July to December
month_order = ['July', 'August', 'September', 'October', 'November', 'December']

# Convert 'month' column to categorical with specified order
final_df['month'] = pd.Categorical(final_df['month'], categories=month_order, ordered=True)

# Group by 'Month' and 'is_holiday' columns and count the occurrences
holiday_counts = final_df.groupby(['month', 'is_holiday'])['date'].nunique().reset_index(name='Count')

# Create a countplot
plt.figure(figsize=(10, 10))
sns.barplot(x='month', y='Count', hue='is_holiday', data=holiday_counts, palette='Set2')
plt.title('Count of Non-Holidays vs Holidays from July to December')
plt.xlabel('Month')
plt.ylabel('Day Count')
plt.xticks(rotation=45)
plt.legend(title='Holiday Status')
plt.show()

The graph titled "Count of Non-Holidays vs Holidays from July to December" provides a clear visualization of the distribution of non-holidays and holidays across the months from July to December. Here's a detailed summary based on the graph:

- **July to December Distribution**: The graph spans from July to December along the x-axis, representing these six months. Each month is divided into two sections: one for non-holidays (green bars) and the other for holidays (orange bars).

- **Non-Holiday Counts**: The green bars, representing non-holidays, consistently reach nearly 30 days for each month. This indicates that there are almost 30 non-holidays in each month, reflecting a regular pattern of workdays or non-holiday periods.

- **Holiday Counts**: In contrast, the orange bars, representing holidays, appear at the bottom of each month and are significantly shorter compared to the green bars. This indicates that the number of holidays is relatively lower compared to non-holidays. 

- **Legend Explanation**: The legend in the top-right corner provides clarity on the color representation:
  - The green color corresponds to "No" holiday status.
  - The orange color corresponds to "Yes" holiday status.

The distribution of holiday days across the months shows variation, with some months having fewer holidays than others. It highlights the prevalence of non-holiday days compared to holidays and allows for easy comparison across the months.

#### **Bivariate Analysis**

##### **Total Capacity of Parking Spots for each Facility**

In [ ]:
# Group by facility name and sum the total parking spots
facility_parking_spots = final_df.groupby('facility_name')['capacity'].first().reset_index()

# Sort the DataFrame by total parking spots in descending order
facility_parking_spots = facility_parking_spots.sort_values(by='capacity', ascending=False)

# Create a bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x='facility_name', y='capacity', data=facility_parking_spots)
plt.xticks(rotation=45, ha='right')
plt.xlabel('Facility Name')
plt.ylabel('Total Parking Spots')
plt.title('Total Parking Spots by Facility (Descending Order)')
plt.tight_layout()
plt.show()

The graph titled "Total Parking Spots by Facility (Descending Order)" offers a comprehensive overview of the distribution of parking spots across different facilities. Here's a detailed summary based on the graph:

- **Facility Distribution**: The x-axis lists the names of various facilities (car parks), providing a clear identification of each entity included in the analysis and the y-axis represents the total number of parking spots, ranging from 0 to 1750. Each bar on the graph corresponds to a specific facility and depicts the total number of parking spots available at that location.

- **Descending Order**: The bars are arranged in descending order, with the facility having the highest number of parking spots positioned at the top of the graph. This arrangement facilitates quick identification of facilities with the most parking spots. **Leppington Car Park** emerges as the facility with the highest number of spots, while **Kiama Car Park** registers the lowest count.

This graph effectively presents the distribution of parking spots across various facilities, enabling stakeholders to identify facilities with the highest and lowest parking capacity at a glance. It serves as a valuable tool for decision-making and resource allocation related to parking management.

In [ ]:
# Create a side-by-side boxplot or violin plot
plt.figure(figsize=(12, 6))

# Side-by-side boxplot
plt.subplot(1, 2, 1)
sns.boxplot(x='time_category', y='parking_availability',
            data=final_df, palette='Set2')
plt.title('Parking Availability Distribution Across Time Intervals')

# Side-by-side violin plot
plt.subplot(1, 2, 2)
sns.violinplot(x='time_category', y='occupancy',
               data=final_df, palette='Set2')
plt.title('Occupancy Rate Distribution Across Time Intervals')

plt.tight_layout()
plt.show()

1. **Parking Availability Distribution Across Time Intervals**

The green bars represent the number of available parking spots during different times of day.
    
- **Night**: There are ample parking spots available during the night.
- **Early Morning**: Availability remains high.
- **Morning**: Still a good number of spots.
- **Afternoon**: Availability starts to decrease slightly.
- **Late Afternoon**: A dip in availability.
- **Evening**: The lowest availability, but still some spots.


2. **Occupancy Rate Distribution Across Time Intervals**

 The violin plots show the distribution of occupancy rates.
    
- **Night**: Occupancy is relatively low, with a wide spread.
- **Early Morning**: Occupancy remains low.
- **Morning**: A peak around the median occupancy.
- **Afternoon**: Occupancy increases, with a narrower spread.
- **Late Afternoon**: Highest occupancy, concentrated around the median.
- **Evening**: Occupancy decreases slightly.

The analysis reveals insights into parking availability and occupancy rates across various time intervals. Parking availability remains relatively consistent throughout the day, with a slight decline observed in the late afternoon and evening. Conversely, occupancy rates peak during the afternoon and late afternoon periods, indicating increased demand for parking spaces during these times. For optimal parking opportunities, it is recommended to target early morning or late afternoon intervals, where a favorable balance between availability and occupancy is likely to be found.

##### **Monthly Average Occupancy per facility**

In [ ]:
# List of unique facility names
facility_names = final_df['facility_name'].unique()

# Set the style of the plot
sns.set_style("whitegrid")

# Calculate the number of rows and columns for subplots
num_facilities = len(facility_names)
num_cols = 4  # Number of columns per row
num_rows = math.ceil(num_facilities / num_cols)  # Round up to the nearest integer

# Create subplots with calculated rows and columns
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(18, num_rows * 5), sharex=True)

# Iterate over each facility name and its corresponding axis
for idx, (facility_name, ax) in enumerate(zip(facility_names, axes.flatten())):
    # Filter data for the current facility
    facility_data = final_df[final_df['facility_name'] == facility_name]
    
    # Group the data by month and calculate the average occupancy total
    facility_month_avg = facility_data.groupby('month')['occupancy'].mean().reset_index()
    
    # Create bar plot for the current facility
    sns.barplot(data=facility_month_avg, x='month', y='occupancy', palette='viridis', ax=ax)
    
    # Set title and labels for the current subplot
    ax.set_title(f'Average Occupancy Total for {facility_name}')
    ax.set_xlabel('Month')
    ax.set_ylabel('Average Occupancy Total')
    ax.set_xticks(range(6))
    ax.set_xticklabels(['July', 'August', 'September', 'October', 'November', 'December'], rotation=45)
    ax.tick_params(axis='x', labelrotation=45)
    ax.grid(True)

# Hide empty subplots if the number of facilities is not a multiple of num_cols
if num_facilities % num_cols != 0:
    for ax in axes.flatten()[num_facilities:]:
        ax.axis('off')

# Adjust layout
plt.tight_layout()
plt.show()


The graph illustrates the average occupancy across the months of July through December for a dataset comprising 28 parking facilities. On the x-axis, we have the months, ranging from July to December, while the y-axis represents the average occupancy level. Each data point on the graph corresponds to the average occupancy recorded for a specific month across all 28 parking facilities.

The observed trend indicates that December exhibits the lowest average occupancy compared to the preceding months of July through November. This decline in occupancy during December can be attributed to several factors:

1. **Holiday Season**: December encompasses the holiday season, characterized by public holidays such as Christmas and New Year's Eve. During this period, many individuals may be on vacation or traveling, resulting in reduced demand for parking facilities and consequently lower occupancy rates.

2. **Reduced Workdays**: December often includes public holidays and reduced workdays, leading to fewer commuters and workers requiring parking spaces. This decrease in daily commuting contributes to a decline in overall occupancy levels across parking facilities.

3. **Travel Plans**: Some individuals may travel during December for vacations or family gatherings, further reducing the local demand for parking facilities and contributing to lower average occupancy rates.

Overall, the combination of holiday-related factors, reduced workdays, school holidays, and altered shopping patterns during December likely contributes to the observed decrease in parking facility occupancy compared to the preceding months.

##### **Average Occupancy by Day of the Week**

In [ ]:
# Order of days of the week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# First subplot: Average Occupancy by Day of the Week
plt.figure(figsize=(10, 5))  # Adjust the figure size as needed
sns.barplot(x='day_of_week', y='occupancy', data=final_df, palette='deep', order=day_order)
plt.title('Average Occupancy by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Average Occupancy')
plt.tight_layout()

# Show the first plot
plt.show()

# Second subplot: Occupancy by Day of the Week per Parking Facility
plt.figure(figsize=(20, 10))  # Adjust the figure size as needed
sns.barplot(x='day_of_week', y='occupancy', data=final_df, palette='viridis', hue='facility_id', order=day_order)
plt.title('Occupancy by Day of the Week per Parking Facility')
plt.xlabel('Day of the Week')
plt.ylabel('Average Occupancy')
plt.legend(title='Parking Facility', loc='upper right', bbox_to_anchor=(1.25, 1))
plt.tight_layout()

# Show the second plot
plt.show()


**Average Occupancy by Day of the Week**

- **Axes**:
    - **Y-Axis**: Represents the average occupancy, ranging from **0** to **350**.
    - **X-Axis**: Lists the days of the week from **Monday** to **Sunday**.

The graph depicts the average occupancy levels across different days, ranging from Monday to Sunday. Tuesday stands out with the highest average occupancy, surpassing 325, while Sunday records the lowest average occupancy, barely surpassing 100. The other days fall between these extremes.



**Occupancy by Day of the Week per Parking Facility**

- **Axes**:
    - **Y-Axis**: Represents the **occupancy level**, ranging from **0** to **900**.
    - **X-Axis**: Lists the days of the week from **Monday** to **Sunday**.

- **Bars**:
    - Each colored bar represents a different **parking facility**, numbered from **6** to **33**.
    - The colors of the bars correspond to specific facilities, as indicated in the legend on the right side of the graph.
    - Some facilities show **high occupancy**, while others have **lower occupancy** levels.

The second graph illustrates occupancy levels across various parking facilities. While some facilities consistently exhibit high occupancy levels, others show lower and more variable occupancy rates across different days of the week.

**Recommendations**
- **Peak Days**: Consider allocating additional resources (staff, maintenance) on days with high occupancy.
- **Facility-Specific Strategies**: Tailor management strategies based on the occupancy patterns of individual facilities.

##### **Average Occupancy Percentage by facility**

In [ ]:
# Calculate the percentage of occupied slots
final_df['OccupancyPercentage'] = (final_df['occupancy'] / final_df['capacity']) * 100

# Group by ParkingLotID and calculate average occupancy percentage
average_occupancy = final_df.groupby('facility_name')['OccupancyPercentage'].mean().reset_index()

# Sort the DataFrame by average occupancy percentage in ascending order
average_occupancy = average_occupancy.sort_values(by='OccupancyPercentage')

# Create a bar plot
plt.figure(figsize=(10, 6))
sns.barplot(y = average_occupancy['facility_name'], x = average_occupancy['OccupancyPercentage'],
            data=final_df, palette='viridis', orient='h')
plt.title('Average Parking Lot Occupancy')
plt.ylabel('Parking Lot Name')
plt.xlabel('Average Occupancy Percentage (%)')
# plt.ylim(0, 100)
plt.show()

The bar graph above represents the average occupancy percentage of various car parks. Here are the details:

**Graph Structure**:
- The x-axis represents the "Average Occupancy Percentage (%)" ranging from 0% to 70%.
- The y-axis lists the names of different car parks. Each bar's length corresponds to the average occupancy percentage of that particular car park.

**Car Park Occupancy**:
- **Perith Combewood At-Grade Car Park** has the lowest occupancy rate, below 10%.
- **Campbelltown Farrow Rd North Car Park** has the highest occupancy rate, close to 70%.

##### **Monthly Average Parking Availability per facility**

In [ ]:
# List of unique facility names
facility_names = final_df['facility_name'].unique()

# Set the style of the plot
sns.set_style("whitegrid")

# Calculate the number of rows and columns for subplots
num_facilities = len(facility_names)
num_cols = 4  # Number of columns per row
num_rows = math.ceil(num_facilities / num_cols)  # Round up to the nearest integer

# Create subplots with calculated rows and columns
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(18, num_rows * 5), sharex=True)

# Iterate over each facility name and its corresponding axis
for idx, (facility_name, ax) in enumerate(zip(facility_names, axes.flatten())):
    # Filter data for the current facility
    facility_data = final_df[final_df['facility_name'] == facility_name]
    
    # Group the data by month and calculate the average parking_availability total
    facility_month_avg = facility_data.groupby('month')['parking_availability'].mean().reset_index()
    
    # Create bar plot for the current facility
    sns.barplot(data=facility_month_avg, x='month', y='parking_availability', palette='viridis', ax=ax)
    
    # Set title and labels for the current subplot
    ax.set_title(f'Average Parking Availability for {facility_name}')
    ax.set_xlabel('Month')
    ax.set_ylabel('Average Parking Availability')
    ax.set_xticks(range(6))
    ax.set_xticklabels(['July', 'August', 'September', 'October', 'November', 'December'], rotation=45)
    ax.tick_params(axis='x', labelrotation=45)
    ax.grid(True)

# Hide empty subplots if the number of facilities is not a multiple of num_cols
if num_facilities % num_cols != 0:
    for ax in axes.flatten()[num_facilities:]:
        ax.axis('off')

# Adjust layout
plt.tight_layout()
plt.show()

The graph above illustrates the monthly average parking availability for each facility, spanning from July through November. 

**Graph structure**:
- x-axis represents months from Jult through December
- y-axis indicates the average parking availability for the corresponding facility during that month. 

There is a noticeable increase in parking availability during December compared to the preceding months. This trend suggests that December generally exhibits higher parking availability across facilities compared to the earlier months. Possible reasons for this could include reduced demand for parking due to holidays, vacations, or changes in usage patterns during the festive season.

##### **Average Parking Availability By Day of the Week**

In [ ]:
# Order of days of the week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# First subplot: Average Parking Availability by Day of the Week
plt.figure(figsize=(10, 5))  # Adjust the figure size as needed
sns.barplot(x='day_of_week', y='parking_availability', data=final_df, palette='deep', order=day_order)
plt.title('Average Parking Availability by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Average Parking Availability')
plt.tight_layout()

# Show the first plot
plt.show()

# Second subplot: Parking Availability by Day of the Week per Parking Facility
plt.figure(figsize=(20, 10))  # Adjust the figure size as needed
sns.barplot(x='day_of_week', y='parking_availability', data=final_df, palette='viridis', hue='facility_id', order=day_order)
plt.title('Parking Availability by Day of the Week per Parking Facility')
plt.xlabel('Day of the Week')
plt.ylabel('Parking Availability')
plt.legend(title='Parking Facility', loc='upper right', bbox_to_anchor=(1.25, 1))
plt.tight_layout()

# Show the second plot
plt.show()

**Average Parking Availability by Day of the Week**.

**Graph Structure**:
- The x-axis represents the "Day of the Week," featuring days from Monday to Sunday.
- The y-axis is labeled as "**Average Parking Availability**" and ranges from 0 to 500.
Each day of the week has a distinct colored bar that reaches up to approximately 500, indicating high parking availability throughout the week.


**Interpretation**:
The graph provides insights into the average parking availability for each day of the week. All bars are almost equal in height, suggesting similar parking availability across all days.
Despite minor fluctuations, parking availability remains consistent throughout the week.

**Parking Availability by Day of the Week per Parking Facility**

**Graph Structure**:
- The x-axis represents the days of the week, starting from **Monday** and ending with **Sunday**.
- The y-axis shows the number of available parking spots, ranging from **0** to approximately **1600**.
- Each day has multiple bars (in various colors) representing data for different parking facilities shown by the legend on the right side which indicates the color corresponding to each facility.

**Observations**:
   - **Saturday** and **Sunday** exhibit some of the highest peaks, indicating high parking availability during these days. Availability varies significantly both between days and across different facilities.

##### **Parking Availability by Parking Facility**

In [ ]:
# Sort the DataFrame by parking availability in descending order
final_df_sorted = final_df.sort_values(by='parking_availability', ascending=False)

# Bar plot
plt.figure(figsize=(8, 8))
sns.barplot(y='facility_name', x='parking_availability',
            data=final_df_sorted, palette='viridis', orient='h')
plt.title('Parking Availability by Parking Facility')
plt.xlabel('Parking Availability')
plt.ylabel('Parking Facility')
plt.show()

**Parking Availability by Parking Facility**

The graph above provides information about the parking availability at various car parks. Here are the details:

**Graph Structure**:
- The x-axis represents the "Parking Availability," ranging from 0 to approximately 1400 available spaces.
- The y-axis lists the names of different car parks.
Each bar corresponds to a specific car park and indicates the number of available parking spaces.

**Car Parks and Availability**:
   - **Leppington Car Park** and **Edmondson Park South Car Park** have the longest bars, suggesting they have the most available spaces.
   - Other car parks, such as **Penrith Combeewood Multi-Level**, **Gosford**, **Kellyville South**, **Warwick Farm**, **Revesby**, and **Bella Vista**, also show significant availability.

##### **Parking Lot Capacity vs. Occupancy**

In [ ]:
# Create a figure with two subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

# Plotting the line plot on the first subplot
sns.lineplot(x='facility_id', y='capacity', data=final_df, label='Capacity', marker='o', ax=axes[0])
sns.lineplot(x='facility_id', y='occupancy', data=final_df, label='Occupancy', marker='o', ax=axes[0])

# Adding labels and title to the first subplot
axes[0].set_xlabel('Parking Lots')
axes[0].set_ylabel('Number of Parking Spots')
axes[0].set_title('Parking Lot Capacity vs. Occupancy')
axes[0].legend()

# Melt the DataFrame for Seaborn
df_melted = pd.melt(final_df[['facility_name', 'capacity', 'occupancy']], id_vars='facility_name', var_name='Category', value_name='Value')

# Creating a multiple bar plot on the second subplot
sns.barplot(y='facility_name', x='Value', hue='Category', data=df_melted, palette='muted', orient='h', ax=axes[1])

# Adding labels and title to the second subplot
axes[1].set_ylabel('Parking Facilities')
axes[1].set_xlabel('Number of Parking Spots')
axes[1].set_title('Parking Lot Capacity vs. Occupancy')
axes[1].legend(title='Category')

# Adjust layout
plt.tight_layout()

# Show the combined plot
plt.show()


**Line Chart**:
- **Blue Line (Capacity)**: Represents the total number of parking spots available (capacity).
- **Orange Line (Occupancy)**: Indicates the actual car occupancy in the parking lots.
Both lines fluctuate over time and generally, the orange line (occupancy) remains below the blue line (capacity), suggesting available spaces in all lots.


**Bar Chart**:
The right graph lists specific car parks along with their capacities and occupancies and each bar represents a parking facility:
- **Blue Bars**: Show the total capacity (number of parking spots).
- **Orange Bars**: Represent the current occupancy levels.
Observations:
- Most car parks have available spaces (difference between blue and orange bars) and some car parks are nearly full.
    

**Recommendations**:
- **Underutilized Lots**:
For car parks with consistently low occupancy, consider:
    - Implementing dynamic pricing to encourage use during off-peak hours.
    - Explore partnerships (with nearby businesses) to increase utilization.

- **Highly Occupied Lots**:
For lots nearing full capacity, consider:
    - Expanding or optimizing management systems to prevent overflow.
    - Offering alternative transportation options (shuttles, public transit) to reduce reliance on personal vehicles.
    - Implementing real-time availability updates to guide drivers to less crowded lots.

##### **Parking Availability by Time Category**

In [ ]:
# Bar plot grouped by time category
plt.figure(figsize=(7, 4))

# Order of time category
time_order = ['Early Morning', 'Morning', 'Afternoon', 'Late Afternoon', 'Evening', 'Night']
sns.barplot(x='time_category', y='parking_availability',
            data=final_df, palette='viridis', order = time_order)
plt.title('Parking Availability by Time Category')
plt.xlabel('Time Category')
plt.ylabel('Parking Availability')
plt.show()

**Parking Availability by time category**

**Graph Structure**:
- The x-axis represents different **time categories**
- The y-axis represents the **number of available parking spots** and the vertical bars indicate the parking availability for each time category.

The graph illustrates the distribution of parking availability across different time categories. Early morning and night exhibit the highest availability, with approximately 700 spots each, followed by evening with around 600 spots. Morning and late afternoon maintain relatively stable availability, with about 300 spots each. The observations suggest potential opportunities for optimizing parking management strategies, such as dynamic pricing during peak hours and providing real-time availability updates to drivers.

####**Multivariate Analysis**

##### **Effect of Holiday on Parking Availability**

In [ ]:
# Create a bar plot with Seaborn
plt.figure(figsize=(7, 5))
sns.barplot(x='month', y='parking_availability', hue='is_holiday', data=final_df, palette='Set2', estimator='mean')

# Adding labels and title
plt.title('Average Parking Availability: Holidays vs Non-Holidays (July to December)')
plt.xlabel('Month')
plt.ylabel('Average Parking Availability')
plt.xticks(rotation=45)

# Display the legend
plt.legend(title='Holiday Status')

# Show the plot
plt.show()

**Average Parking Availability: Holidays vs Non-Holidays (July to December)**

**Graph Structure**:
- The title of the graph is "**Average Parking Availability: Holidays vs Non-Holidays (July to December)**."
- The x-axis represents the months from **July** to **December** with two sets of bars for each month:
    - **Orange Bars**: Represent holidays.a
    - **Green Bars**: Represent non-holidays.
- The y-axis represents the **average parking availability**, ranging from 0 to slightly above 600.

**Observations**:
- For most months (July to November), parking availability is higher during **non-holidays**. However, in **December**, parking availability is higher during **holidays**. This observation suggests the need to adjust parking management strategies to accommodate holiday schedules, optimize resources during peak holiday periods, and continually monitor trends to ensure sufficient availability.

##### **Correlation Heatmap**

In [ ]:
# Filter only numeric columns
numeric_columns = ['parking_availability', 'capacity', 'occupancy',  'longitude', 'latitude']

# Select numeric columns from the DataFrame
numeric_data = final_df[numeric_columns]

# Create a correlation matrix
correlation_matrix = numeric_data.corr()

# Set up the matplotlib figure
plt.figure(figsize=(6, 4))

# Create a heatmap using Seaborn
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# Set the title
plt.title('Correlation Heatmap for Numeric Columns')

# Show the plot
plt.show()


Breakdown:

**Graph Structure**:
- The x-axis and y-axis list the following variables:
    - **parking_availability**
    - **capacity**
    - **occupancy**
    - **longitude**
    - **latitude**

- The heatmap colors indicate the strength of correlation:
    - Red (1): Strong positive correlation
    - Blue (-0.2): Negative correlation

The heatmap illustrates correlations between various factors in the dataset. Strong positive correlations exist between parking availability and capacity, as well as between occupancy and capacity. Additionally, latitude and longitude exhibit a moderate positive correlation. Notably, parking availability shows a negative correlation with latitude. These insights can guide parking management decisions, indicating that areas with larger capacity tend to have higher availability.

In [ ]:
# Create a pivot table for heatmap
heatmap_data = final_df.pivot_table(
    values='parking_availability', index='day_of_week', columns='time_category', aggfunc='mean')

# Define the order of days of the week
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Define the order of time categories
time_order = ['Morning', 'Afternoon', 'Evening', 'Night']

# Create a heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(heatmap_data.loc[days_order, time_order], annot=True, cmap='viridis', fmt='.1f', linewidths=.5)
plt.title('Overall Parking Availability Trends')
plt.xlabel('Time Category')
plt.ylabel('Day of the Week')
plt.show()

##### **Time Series Decomposition**

In [ ]:
# Assuming final_df contains the data with the 'date' column as the index

# List of unique facility names
facility_names = final_df['facility_name'].unique()

# # Set the 'date' column as the index for the entire DataFrame
# final_df.set_index('date', inplace=True)

# Iterate through each facility
for facility_name in facility_names:
    # Filter data for the current facility
    facility_data = final_df[final_df['facility_name'] == facility_name]
    
    # Resample the data to daily frequency and fill missing values
    facility_data = facility_data.resample('D').mean().fillna(method='ffill')
    
    # Perform seasonal decomposition
    decomposition = seasonal_decompose(facility_data['parking_availability'], model='additive', period=7)  # Assuming weekly seasonality
    
    # Plot the decomposition results
    plt.figure(figsize=(12, 8))
    plt.suptitle(f"Seasonal Decomposition for {facility_name}", fontsize=16)

    plt.subplot(411)
    plt.plot(facility_data['parking_availability'], label='Original')
    plt.legend()

    plt.subplot(412)
    plt.plot(decomposition.trend, label='Trend')
    plt.legend()

    plt.subplot(413)
    plt.plot(decomposition.seasonal, label='Seasonality')
    plt.legend()

    plt.subplot(414)
    plt.plot(decomposition.resid, label='Residuals')
    plt.legend()

    plt.tight_layout()
    plt.show()


Let's delve deeper into the time series decomposition graph and explore each component:

1. **Original Component**:
   - The **original component** represents the **raw data** without any adjustments. In this case, it appears to be a dataset that varies over time.
   - Looking at the graph, we notice several oscillations (ups and downs) along with an overall declining trend. These fluctuations could be due to various factors, such as seasonality, external events, or underlying patterns.

2. **Trend Component**:
   - The **trend component** is obtained by **smoothing out** the original data to reveal the **long-term behavior** or direction.
   - In your graph, the trend shows a **gradual decline** over the observed period (from July 2023 to January 2024). This decline might indicate a consistent downward movement in the underlying phenomenon being measured.

3. **Seasonality Component**:
   - The **seasonality component** captures **recurring patterns** that occur at regular intervals (e.g., daily, monthly, or yearly).
   - From the graph, we can see periodic peaks and valleys. These could correspond to seasonal effects, such as monthly fluctuations, holidays, or other cyclic events.

4. **Residuals Component**:
   - The **residuals component** represents the **unexplained variations** in the data after removing the trend and seasonality.
   - These residuals are essentially the **noise** or irregularities left behind. They could result from random fluctuations, measurement errors, or other factors not accounted for by the trend and seasonality.

**Conclusion**:
- The declining trend suggests that there might be an underlying reason for the overall decrease in the observed phenomenon.
- Seasonality could provide insights into recurring patterns. For example, if this data represents sales, the peaks might coincide with holiday seasons.
- Analyzing the residuals can help identify anomalies or unexpected deviations from the expected behavior.

####**Next Steps**

1. To implement a real-time data acquisition pipeline to continuously update the model with the latest parking occupancy data.

2. To conduct pilot tests and gather feedback from end-users to assess the usability and effectiveness of the parking prediction system in real-world scenarios.

3. To deploy the finalized parking spot predictor in urban areas, collaborating with city authorities and parking management companies to integrate it into existing infrastructure and promote widespread adoption.

### **5. Modelling**

Starting off by instantiating a model object

In [ ]:
models = Modelling(data_analysis)

Starting off, a new variable `data` will be used to represent the dataframe being used to create the models.

In [ ]:
data = models.carpark_history

In [ ]:
def g(df):
    df["time"] = df["time"].astype("str")
    df["timestamp"] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M:%S')
    return df

modelling_data = g(data)
#setting timestamp as the index
modelling_data.set_index("timestamp",  inplace=True)

In [ ]:
#dropping columns we dont need
modelling_data.drop(["ParkID", "zone_id", "date", "time"], axis= 1,inplace= True)
modelling_data

In [ ]:
modelling_data.columns

In [ ]:
plotting_data = modelling_data.copy()
plotting_data.drop(columns=['facility_name', 'total_parking_spots', 'occupancy_total',
    'day_of_week', 'time_category', 'z_score'], inplace=True, axis=1)

In [ ]:
plotting_data

Plotting the total parking availability for time and day of the week

In [ ]:
#plotting the parking availability trend
plotting_data.plot(figsize = (15, 8))
plt.title("parking pattern over time")
plt.xlabel('Time')
plt.ylabel("total_parking_availability")
plt.show()

Our time series data has high frequency or irregular time intervals, we considered resampling to a lower frequency to obtain a more meaningful plot. Resampling allows you to aggregate data over specified time intervals, in our case we shall resample the data down to day, week and month

We wrote a function to help us plot the patterns

In [ ]:
#helper function for plotting patterns
def parking_pattern(data, size):
    data.plot(figsize = (15, 8))
    plt.title(f"parking pattern on a {size} basis")
    plt.xlabel('Time')
    plt.ylabel("total_parking_availability")
    plt.show()

1. Daily pattern of parking availability

In [ ]:
daily_pattern = plotting_data.resample("D").mean()
parking_pattern(daily_pattern, "daily")

2. Weekly pattern of parking availability

In [ ]:
weekly_pattern = plotting_data.resample("W").mean()
parking_pattern(weekly_pattern, "weekly")

3. Monthly parking availability pattern

In [ ]:
monthly_pattern = plotting_data.resample("M").mean()
parking_pattern(monthly_pattern, "monthly")

Checking for stationarity. A time series is considered stationary if its statistical properties, such as mean, variance, and autocorrelation, do not change over time. In simple words, the time series data shows consistent behavior and does not exhibit trends or seasonality.

In [ ]:
#importing adfuller test from statsmodels
from statsmodels.tsa.stattools import adfuller
def stationarity_test(dataset):
    ad_test = adfuller(dataset, autolag = "AIC")
    print("1. ADF: ",ad_test[0])
    print("2. p-value: ",ad_test[1])
    print("3. Number of lags: ", ad_test[2])
    print("4. Number of obeservations used for ADF regression and critical values calculation: ", ad_test[3])
    print("5. Critical values: ")
    

instantiating the autoarima model for our time series problem

In [ ]:
daily_pattern

Now that we already know the behavior of our data, the model to use and and the order to specify we will go ahead and train a model and make some few predictions. We are also going to split our data into train, test and validation

In [ ]:
#splitting data
train_data, test_data = train_test_split(daily_pattern, test_size= 0.3, random_state= 42)

In [ ]:
#instantiating ARIMA model
arima_model = ARIMA(train_data, order = (2, 1, 2))
arima_model = arima_model.fit()
arima_model.summary()

We will also do a prediction to see how well our model has learnt. For the predict method, you need to pass in the start and end parameters for the period you want to predict

In [ ]:
#specifying start and end
start = len(train_data)
end = len(train_data) + len(test_data)-1
predictions = arima_model.predict(start = start, end= end, type = "level")
predictions.index = plotting_data.index[start:end+1]
print(round(predictions, 2))


In [ ]:
predictions.plot(legend =True)
# daily_pattern["parking_availability"].plot(legend= True)

In [ ]:
print(daily_pattern.mean())

In [ ]:
#getting the mse and rmse
pred_rmse = sqrt(mean_squared_error(predictions, test_data))
print(pred_rmse)

The rmse range is greater than the mean of the data, meaning that for every prediction we make we are getting a value 160.2 further away from the actual result. This means that our model is not performing well and needs improvement 

####**Making Future Predictions**

For this to take place effectively we will train the model on the whole dataset

In [ ]:
# arima_model2 = ARIMA(plotting_data, order= (2,1,2))
# arima_model2 = arima_model2.fit()

In [ ]:
# #setting up future dates 
# future_dates = pd.date_range(start= "2024-01-01", end = "2024-01-31")
# #making prediction
# future_preds = arima_model2.predict(start = len(plotting_data), end = len(plotting_data)+ 30, type = "levels")
# future_preds.index = future_dates
# print(future_preds)

Viewing the predictions from the ARIMA model, we can see there is similarity in the values except for the decimal values. This indicates our model has not captured most of the seasonality and trend component.Our model maybe also overfitting to the training data or it may be that the series does not have a strong enough seasonal component.

Having also many facilities makes it hard to interpret the prediction for each facility. To overcome this problem, we are going to group our data using facility name and then model time series for each facility in our dataset. This will ease prediction of a parking availability for a given facility, date and time

In [ ]:
grouped_data = dddata.copy()
grouped_data.drop(columns=["ParkID", "zone_id"], axis= 1,inplace= True)

grouped_data

In [ ]:
# import prophet
# # Create the ColumnTransformer
# ohe_pipeline = Pipeline(steps=[('enc', OneHotEncoder(drop="first", sparse=False))])
# transformer = ColumnTransformer(transformers=[("ohe", ohe_pipeline, categorical_columns)], remainder="passthrough")

# # Group data by facility_name
# grouped_data = modelling_data.groupby("facility_name")

# # Loop through each facility until the last one
# for facility, facility_data in grouped_data:
#     # Apply OHE within this facility's data
#     transformed_data = transformer.fit_transform(facility_data)

#     # Reconstruct DataFrame with encoded columns
#     transformed_df = pd.DataFrame(transformed_data, columns=transformer.get_feature_names_out())

#     # Select columns for Prophet (assuming 'ds' for datetime, 'occupancy_total' for target)
#     model_data = transformed_df[['ohe__day_of_week_Monday', 'ohe__day_of_week_Saturday',
#     'ohe__day_of_week_Sunday', 'ohe__day_of_week_Thursday',
#     'ohe__day_of_week_Tuesday', 'ohe__day_of_week_Wednesday',
#     'ohe__time_category_Early Morning', 'ohe__time_category_Evening',
#     'ohe__time_category_Late Afternoon', 'ohe__time_category_Morning',
#     'ohe__time_category_Night', 'remainder__total_parking_spots',
#     'remainder__occupancy_total', 'remainder__parking_availability',
#     'remainder__z_score', 'date', 'time']] 

#     # Create and fit Prophet model
#     model = prophet.Prophet()
#     model.fit(model_data)

#     # Make predictions for a specified future period (e.g., next week)
#     future = model.make_future_dataframe(periods=prediction_period)
#     forecast = model.predict(future)

#     # Analyze and store results for this facility (optional)
#     # ...

#     # Check if this is the last facility
#     if next(iter(grouped_data), None) is None:  # Check if there are no more facilities
#         print("Processed predictions for all facilities")
#         break

# # Further analysis or visualization using predictions/models for each facility (optional)
# # ...

We will create a class for the Prophet that iterates over grouped data does time series using given columns and allows us to make predictions on future data.

First lets preprocess our categorical/string dtype columns

In [ ]:
from datetime import datetime
#function to conver to datetime
def to_datetime(x):
    return datetime.strptime(x,  "%Y-%m-%d %H:%M:%S")

pprophet_data = pd.read_csv("modelling_data.csv", parse_dates= [["date", "time"]], date_parser= to_datetime)

In [ ]:
#setting datetime as index
prophet_data = pprophet_data.drop(["day_of_week", "time_category", "ParkID", "zone_id", "z_score"], axis= 1)
prophet_data = prophet_data.set_index("date_time").groupby("facility_name").resample("D").mean()
#inspecting our data
prophet_data.head()

We have two index columns in our grouped dataframe, we will reset the index, return date_time as the index and plot and do some visualizations on our facilities

In [ ]:
#resetting index
prophet_data = prophet_data.reset_index()

Plotting all trends in one plot

In [ ]:
def plot_facilities(data, facilities, columns=["total_parking_spots", "occupancy_total", "parking_availability"], figsize=(15, 5)):
    # Set the 'date_time' column as the index
    data.set_index("date_time")

    # Filter data for the specified facilities
    filtered_data = data[data['facility_name'].isin(facilities)]

    for facility in facilities:
        facility_data = filtered_data.query(f"facility_name == '{facility}'")

        # Create a new figure for each facility
        fig, ax = plt.subplots(figsize=figsize)
        
        # Plot the data for the current facility
        facility_data[columns].plot(ax=ax, label=facility)

        # Set title, labels, and legend
        ax.set_title(f"Time Series Data for {facility}")
        ax.set_xlabel("Date and Time")
        ax.set_ylabel("Values")
        ax.legend(loc="upper left", bbox_to_anchor=(1, 1))

        plt.show()

# Example usage
facilities_to_plot = ["Bella Vista Car Park", "West Ryde Car Park", "Narrabeen Car Park", "Campbelltown Farrow Rd North Car Park " ,
                    "Warwick Farm Car Park", "Warriewood Car Park", "Tallawong P3 Car Park ", "Tallawong P3 Car Park", "Tallawong P2 Car Park",
                    "Tallawong P1 Car Park", "Sutherland East Parade Car Park", "St Marys Car Park", "Schofields Car Park ", "Revesby Car Park",
                    "Penrith Combewood Multi-Level Car Park", "Penrith Combewood At-Grade Car Park", "Leppington Car Park", "Kiama Car Park ",
                    "Kellyville South Car Park", "Kellyville North Car Park", "Hornsby Jersey St Car Park", "Hills Showground Car Park", "Gosford Car Park ",
                    "Gordon Henry St North Car Park", "Edmondson Park South Car Park", "Dee Why Car Park", "Cherrybrook Car Park", "Campbelltown Hurley Street South Car Park "]
plot_facilities(prophet_data, facilities_to_plot, figsize=(15, 5))

Voila! We can see our parking facilities have different seasonalities and trends. We can not generalize this and build one time series model. What we will do is to create separate models for each of the facility, which will allow us to make accurate predictions and forecasting.

In [ ]:
prophet_data

##### **Prophet Model**

Prophet is a time series forecasting tool developed by Facebook that utilizes an additive model to capture seasonality, trends, and holidays, providing a user-friendly approach for accurate and efficient predictions. # We will create a final dataframe that has the date_time column and the columns we will need for our analysis. Prophet model expects data in a particular format; it expects the name of date column to be "ds" and the target variable as y # Before everything we will create a prophet pipeline

In [ ]:
#creating a pipeline
prophet_pipeline = Pipeline(steps=[("prophet", prophet.Prophet(interval_width = 0.95))])

In [ ]:
final_prophet_data = prophet_data[["facility_name", "date_time", "total_parking_spots","occupancy_total", "parking_availability"]].rename({"date_time": "ds", "parking_availability": "y"}, axis = "columns")

final_prophet_data

Grouping by facility names to allow us to know all the stations and model different time series

In [ ]:
facility_names = final_prophet_data.groupby("facility_name")
facility_names.head()

Splitting our data into train, test and validation sets

In [ ]:
# for facility in facility_names.groups:
#     group = facility_names.get_group(facility)
#     #train data
#     train = group[(group["ds"]>= "2023-12-01") & (group["ds"]<= "2023-12-16")]
#     #test_data
#     test = group[(group["ds"]>= "2023-12-17") & (group["ds"]<= "2023-12-31")]
    
# print(train.shape)
# print(test.shape)

Creating an empty dataframe so we can build a forecast for the whole 28 facilities

In [ ]:
#creating an empty dataframe
target = pd.DataFrame()
# Looping over facilities to perform forecasting
for facility in facility_names.groups:
    group = facility_names.get_group(facility)
    model = prophet.Prophet(interval_width=0.95)
    
    # Fit the model to the data
    model.fit(group)
    
    # Create a future DataFrame
    future = model.make_future_dataframe(periods=31)
    
    # Generate forecasts
    forecast = model.predict(future)
    
    # Plot the forecast
    model.plot(forecast)
    
    # Rename columns in the forecast DataFrame
    forecast = forecast.rename(columns={"yhat": f"yhat_{facility}", "ds": "ds_forecast"})
    
    # Drop unnecessary columns before merging
    forecast = forecast[["ds_forecast", f"yhat_{facility}"]]
    
    # Merge with updated column names
    target = pd.merge(target, forecast.set_index("ds_forecast"), how="outer", left_index=True, right_index=True)

The blackpoints on the plots are the actual parking_availability and the blue points are the predicted values. The light blue background represents the confidence interval which in our case is set to 95%

In [ ]:
target

In [ ]:
final_target = target[["yhat_"+facility for facility in facility_names.groups.keys()]]

final_target

The dataframe above contains the forecast of the next 1 month based on the previous data the model has learned. We will plot the actual values versus the predicted values to see how well our model is performing.

In [ ]:
#facilities names
facility_names = ["Bella Vista Car Park", "West Ryde Car Park", "Narrabeen Car Park",
                    "Warwick Farm Car Park", "Warriewood Car Park","Tallawong P2 Car Park",
                    "Tallawong P1 Car Park", "Sutherland East Parade Car Park", "St Marys Car Park", "Revesby Car Park",
                    "Penrith Combewood Multi-Level Car Park", "Penrith Combewood At-Grade Car Park", "Leppington Car Park",
                    "Kellyville South Car Park", "Kellyville North Car Park", "Hornsby Jersey St Car Park", "Hills Showground Car Park",
                    "Gordon Henry St North Car Park", "Edmondson Park South Car Park", "Dee Why Car Park", "Cherrybrook Car Park"]

# Loop through facility names
for facility in facility_names:
    # Get actual data for the facility from the original dataset
    actual_data = final_prophet_data.set_index("ds").query(f"facility_name == '{facility}'")["y"]

    # Get predicted data for the facility from the target DataFrame
    predicted_data = target[f"yhat_{facility}"]

    # Concatenate actual and predicted data
    combined_data = pd.concat([actual_data, predicted_data], axis=1)

    # Plot the actual vs predicted data 
    fig, ax = plt.subplots(figsize=(15, 5))
    combined_data.plot(ax=ax, title=f"Actual vs Predicted for {facility}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Values")
    ax.legend(["Actual", "Predicted"])

From the plots we can see our model recognizes the seasonality of our data although it doesn't recognize the peaks and this can suggest doing a multivariate analysis to incorporate other exogenous(external) variables such as holidays and weather to see if our model will recognize the peaks

####**Evaluating the Model**

The metrics used to evaluate timeseries  forecasting models are: MAE (Mean Absolute Error), RMSE (Root Mean Squared Error) and MAPE (Mean Absolute Percentage Error). We will use these metrics to evaluate our model

##### **Multivaruate Prophet Modelling**

As we saw in our initial model that the peaks couldn't be recognized when making predictions, we will go ahead and build another model that incorporates other external variables to see if the model will improve on realising the points it didn't recognize earlier

In [ ]:
#our original dataframe
pprophet_data

In [ ]:
#setting datetime as index
second_prophet_data = pprophet_data.drop(["ParkID", "zone_id", "z_score", "time_category"], axis= 1)
#getting dummies
dummies_df = pd.get_dummies(second_prophet_data[['day_of_week']],
                            columns=["day_of_week"], prefix= ["day"])
#converting dummies to numerical
dummies_df = dummies_df.astype(int)
#joining the dummies and original df
final_df = pd.concat( [second_prophet_data, dummies_df] ,axis=1)
#dropping cat columns
final_df.drop( columns = "day_of_week", axis= 1, inplace = True)

#inspecting our final df
final_df.head()

Now we have all columns in a format we can work with in machine learning.

Lets resample the data into daily observations for easy handling

In [ ]:
#setting date to datetime
final_df['date_time'] = pd.to_datetime(final_df['date_time'])

# Set the datetime as index
final_df.set_index('date_time', inplace=True)
#resampling our data
final_data = final_df.groupby("facility_name").resample("D").mean()
#reset index
final_data = final_data.reset_index()

Renaming our columns in order to fit the prophet model

In [ ]:
#renaming columns
col_mapper  = {"date_time": "ds", "parking_availability": "y"}
#rename
final_data.rename(columns=col_mapper, inplace=True)
#inspecting
final_data.head()

Splitting our data into train and test

In [ ]:
# train= final_data[(final_data['ds'] >= '2023-06-01') & (final_data['ds'] <= '2023-09-31')] 
# test= final_data[(final_data['ds'] > '2023-09-31')]

Instantiating prophet model so we can start modelling

In [ ]:
#grouping by facility in order to get predictions for each
parking_lots = final_data.groupby("facility_name")

In [ ]:
# Creating an empty dataframe
second_target = pd.DataFrame()

# Looping over facilities to perform forecasting
for facility in parking_lots.groups:
    group = parking_lots.get_group(facility)
    
    # Instantiate the model
    prophet_model = prophet.Prophet(interval_width=0.95, daily_seasonality= False)
    
    #Adding regressors to the model
    for col in dummies_df.columns:
        prophet_model.add_regressor(col)
    
    # Fit the model to the data
    prophet_model.fit(group)
    
    # Create a future DataFrame with regressors
    future = prophet_model.make_future_dataframe(periods=31)
    
    for col in dummies_df.columns:
        future[col] = dummies_df[col]
    
    # Generate forecasts
    forecast = prophet_model.predict(future)
    final_data
    # Plot the forecast
    prophet_model.plot(forecast)
    
    # Rename columns in the forecast DataFrame
    forecast = forecast.rename(columns={"yhat": f"yhat_{facility}", "ds": "ds_forecast"})
    
    # Drop unnecessary columns before merging
    forecast = forecast[["ds_forecast", f"yhat_{facility}"]]
    
    # Merge with updated column names
    second_target = pd.merge(second_target, forecast.set_index("ds_forecast"), how="outer", left_index=True, right_index=True)


Plotting actual versus predicted

In [ ]:
#facilities names
facility_names = ["Bella Vista Car Park", "West Ryde Car Park", "Narrabeen Car Park",
                    "Warwick Farm Car Park", "Warriewood Car Park","Tallawong P2 Car Park",
                    "Tallawong P1 Car Park", "Sutherland East Parade Car Park", "St Marys Car Park", "Revesby Car Park",
                    "Penrith Combewood Multi-Level Car Park", "Penrith Combewood At-Grade Car Park", "Leppington Car Park",
                    "Kellyville South Car Park", "Kellyville North Car Park", "Hornsby Jersey St Car Park", "Hills Showground Car Park",
                    "Gordon Henry St North Car Park", "Edmondson Park South Car Park", "Dee Why Car Park", "Cherrybrook Car Park"]

# Loop through facility names
for facility in facility_names:
    # Get actual data for the facility from the original dataset
    actual_data = final_data.set_index("ds").query(f"facility_name == '{facility}'")["y"]

    # Get predicted data for the facility from the target DataFrame
    predicted_data = second_target[f"yhat_{facility}"]

    # Concatenate actual and predicted data
    combined_data = pd.concat([actual_data, predicted_data], axis=1)

    # Plot the actual vs predicted data 
    fig, ax = plt.subplots(figsize=(15, 5))
    combined_data.plot(ax=ax, title=f"Actual vs Predicted for {facility}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Values")
    ax.legend(["Actual", "Predicted"])

Evaluating the model using metrics

In [ ]:
# mse = mean_squared_error(final_data['y'], forecast["yhat"][-len(actual_data):])
# rmse = np.sqrt(mse)
# mae = mean_absolute_error()

##### **LSTM**

In [ ]:
#variables
sequence_length = 5
features = 5
epochs = 15
batch_size = 32
# to scale
columns_to_scale = ['total_parking_spots', 'occupancy_total', 'y']
#Extract columns to scale
to_scale = final_data[columns_to_scale]
#MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
# Fit and transform 
scaled_data = scaler.fit_transform(to_scale)
# Create a DataFrame 
scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)
# Add scaled columns back to the original DataFrame
final_data[columns_to_scale] = scaled_df

# Display the result
final_data.head()

Now our data is ready for Neural Networks training. First we are going to set the datetime as the index.

In [ ]:
#setting ds as index
final_data.set_index("ds", inplace= True)

final_data

##### **XGBoost for Time Series**

XGBoost for time series involves transforming time-related features and historical observations into a structured input for the model. Incorporating relevant temporal information can enhance its ability to capture patterns, trends, and seasonality, improving forecasting accuracy. Experimentation with feature engineering and model parameters is crucial for optimizing performance in time series applications.

In [ ]:
#our initial dataframe
modelling_data.head()

Since XGBOOST enjoys working with incorporated relevant temporal features, we are going to feature engineer our timestamp column by creating new columns that capture the day of the week the hour of the day and minute

In [ ]:
#resetting index
xgboost_data = modelling_data.reset_index()
#inspecting the df
xgboost_data.head()

In [ ]:
#feature engineering timestamp
xgboost_data["year"] = xgboost_data["timestamp"].dt.year
xgboost_data["month"] = xgboost_data["timestamp"].dt.month
xgboost_data["hour"] = xgboost_data["timestamp"].dt.hour
xgboost_data["minute"] = xgboost_data["timestamp"].dt.minute
#inspecting df
xgboost_data.head()

In [ ]:
vanilla_data =  xgboost_data.drop(["day_of_week", "time_category", "z_score"], axis= 1)
vanilla_data = vanilla_data.set_index("timestamp")


In [ ]:
xg_dummies = pd.get_dummies(xgboost_data[['day_of_week', 'time_category']],
                            columns=["day_of_week", "time_category"], prefix= ["day_of_the_week", "time_period"],
                            sparse= False, drop_first= True)
#converting dummies to numerical
xg_dummies = xg_dummies.astype(int)
#joining the dummies and original df
xg_encoded = pd.concat( [xgboost_data, xg_dummies] ,axis=1)
#dropping cat columns
xg_encode = xg_encoded.drop(["time_category", "day_of_week", "z_score"], axis= 1)
xg_encoded = xg_encoded.set_index("timestamp")

In [ ]:
vanilla_data =  xgboost_data.drop(["day_of_week", "time_category", "z_score"], axis= 1)
vanilla_data = vanilla_data.set_index("timestamp")

xg_grouped = vanilla_data.groupby("facility_name")

import xgboost as xgb
from datetime import timedelta

# Dictionary to store XGBoost models for each parking lot
xg_models = {}

# Train a model for each parking lot
for facility_name, group_data in xg_grouped:
    # Split the data into features and target variable
    X = group_data.drop(columns=["parking_availability", "facility_name"], axis=1)
    y = group_data["parking_availability"]

    # Create an XGBoost model
    xg_model = xgb.XGBRegressor(objective='reg:squarederror')

    # Train the model
    xg_model.fit(X, y)

    # Make predictions on the entire dataset
    group_data["prediction"] = xg_model.predict(X)

    # Evaluate the model
    mse = mean_squared_error(y, group_data["prediction"])
    rmse = sqrt(mse)
    mae = mean_absolute_error(y,group_data["prediction"])
    print(f'Mean Squared Error for {facility_name}: {mse}')
    print(f'Root Mean Squared Error for {facility_name}: {rmse}')
    print(f'Mean Absolute Error for {facility_name}: {mae}')
    # Store the trained model in the dictionary
    xg_models[facility_name] = xg_model

    # Plot actual vs predicted values
    plt.figure(figsize=(10, 5))
    plt.plot(group_data.index, y, label='Actual', marker='o')
    plt.plot(group_data.index, group_data["prediction"], label='Predicted', marker='o')
    plt.title(f'Actual vs Predicted for {facility_name}')
    plt.xlabel('Date')
    plt.ylabel('Parking Availability')
    plt.legend()
    plt.show()

    
    # #prediction_function
    # def generate_features_for_next_31_days(future_dates):
    #     future_X = pd.DataFrame(index=future_dates)

    #     # Example: Generating random values for demonstration purposes
    #     future_X["total_parking_spots"] = [10, 12, 15, 18, 20, 22, 25, 28, 30, 32, 35, 38, 40, 42, 45, 48, 50, 52, 55, 58, 60, 62, 65, 68, 70, 72, 75, 78, 80, 82, 85]
    #     future_X["occupancy_total"] = [5, 8, 10, 12, 15, 18, 20, 22, 25, 28, 30, 32, 35, 38, 40, 42, 45, 48, 50, 52, 55, 58, 60, 62, 65, 68, 70, 72, 75, 78, 80, 82]
    #     future_X["year"] = [date.year for date in future_dates]
    #     future_X["month"] = [date.month for date in future_dates]
    #     future_X["hour"] = [date.hour for date in future_dates]
    #     future_X["minute"] = [date.minute for date in future_dates]
    #     future_X["day"] = [date.day for date in future_dates]

    #     return future_X

    # # Make predictions for the next 31 days
    
    # future_dates = group_data.index + timedelta(days=31)
    # future_X = generate_features_for_next_31_days(future_dates)  # You need to implement this function
    # future_predictions = xg_model.predict(future_X)

    # # Plot future predictions
    # plt.figure(figsize=(15, 6))
    # plt.plot(future_dates, future_predictions, label='Future Predictions', marker='o')
    # plt.title(f'Future Predictions for {facility_name}')
    # plt.xlabel('Date')
    # plt.ylabel('Parking Availability')
    # plt.legend()
    # plt.show()


Our model is learning too perfectly from the training data such that we cannot see the plot of actual data points because they have been covered by the predicted values.

We will add some few other columns and penalties to see if it will improve

In [ ]:
# Define the parameter grid for GridSearchCV
param_grid = { 
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 5, 6, 7],
    "gamma": [0, 0.1, 0.2],
    "lambda": [0, 0.01,  0.1],
    "alpha": [0, 0.01, 0.1, 0.2],
    "n_estimators": [20, 40, 50]
}

# # Dictionary to store XGBoost models for each parking lot
# xg_models = {}

# # Train a model for each parking lot
# for facility_name, group_data in xg_grouped:
#     # Split the data into features and target variable
#     X = group_data.drop(columns=["parking_availability", "facility_name"], axis=1)
#     y = group_data["parking_availability"]

#     # Create an XGBoost model
#     xg_model = xgb.XGBRegressor(objective='reg:squarederror')

#     # Perform GridSearchCV
#     grid_search = GridSearchCV(xg_model, param_grid=param_grid, cv=3, n_jobs=-1)
#     grid_search.fit(X, y)

#     # Get the best hyperparameters
#     best_params = grid_search.best_params_
#     print(f'Best Hyperparameters for {facility_name}: {best_params}')

#     # Use the best model from GridSearchCV
#     best_model = grid_search.best_estimator_

#     # Train the model
#     best_model.fit(X, y)

#     # Make predictions on the entire dataset
#     group_data["prediction"] = best_model.predict(X)

#     # Evaluate the model
#     mse = mean_squared_error(y, group_data["prediction"])
#     rmse = sqrt(mse)
#     mae = mean_absolute_error(y, group_data["prediction"])
#     print(f'Mean Squared Error for {facility_name}: {mse}')
#     print(f'Root Mean Squared Error for {facility_name}: {rmse}')
#     print(f'Mean Absolute Error for {facility_name}: {mae}')

#     # Store the trained model in the dictionary
#     xg_models[facility_name] = best_model

#     # Plot actual vs predicted values
#     plt.figure(figsize=(10, 5))
#     plt.plot(group_data.index, y, label='Actual', marker='o')
#     plt.plot(group_data.index, group_data["prediction"], label='Predicted', marker='o')
#     plt.title(f'Actual vs Predicted for {facility_name}')
#     plt.xlabel('Date')
#     plt.ylabel('Parking Availability')
#     plt.legend()
#     plt.show()

Now that we have our best parameters for the Xgboost model. We are going to build final models for each time series and make predictions with it.

In [ ]:
# # Define the parameter grid for GridSearchCV
# param_grid = { 
#     "learning_rate": [0.01, 0.1, 0.2],
#     "max_depth": [3, 5, 6, 7],
#     "gamma": [0, 0.1, 0.2],
#     "lambda": [0, 0.01,  0.1],
#     "alpha": [0, 0.01, 0.1, 0.2],
#     "n_estimators": [20, 40, 50]
# }
# #class XGBOOST
# class XGBoostTimeSeriesModel:
#     def __init__(self, param_grid, target_variable='parking_availability', cv=3, n_jobs=-1):
#         self.param_grid = param_grid
#         self.target_variable = target_variable
#         self.cv = cv
#         self.n_jobs = n_jobs
#         self.models = {}

#     def _get_features_target(self, group_data):
#         X = group_data.drop(columns=[self.target_variable, "facility_name"], axis=1)
#         y = group_data[self.target_variable]
#         return X, y

#     def train_models(self, xg_grouped):
#         for facility_name, group_data in xg_grouped:
#             X, y = self._get_features_target(group_data)

#             # Create an XGBoost model
#             xg_model = xgb.XGBRegressor(objective='reg:squarederror')

#             # Perform GridSearchCV
#             grid_search = GridSearchCV(xg_model, param_grid=self.param_grid, cv=self.cv, n_jobs=self.n_jobs)
#             grid_search.fit(X, y)

#             # Get the best hyperparameters
#             best_params = grid_search.best_params_
#             # print(f'Best Hyperparameters for {facility_name}: {best_params}')

#             # Use the best model from GridSearchCV
#             best_model = grid_search.best_estimator_

#             # Train the model
#             best_model.fit(X, y)

#             # Store the trained model in the dictionary
#             self.models[facility_name] = best_model

#     def predict_and_plot(self, xg_grouped):
#         for facility_name, model in self.models.items():
#             # Get the last available data for each facility
#             last_data = xg_grouped.get_group(facility_name).tail(1).drop(columns=["parking_availability", "facility_name"])

#             # Make predictions for the next time period
#             facility_predictions = model.predict(last_data)

#             # Plot actual vs predicted values
#             group_data = xg_grouped.get_group(facility_name)
#             plt.figure(figsize=(10, 5))
#             plt.plot(group_data.index, group_data[self.target_variable], label='Actual', marker='o')
#             plt.plot(group_data.index, group_data["prediction"], label='Predicted', marker='o')
#             plt.title(f'Actual vs Predicted for {facility_name}')
#             plt.xlabel('Date')
#             plt.ylabel('Parking Availability')
#             plt.legend()
#             plt.show()

# # Example usage:
# xg_model = XGBoostTimeSeriesModel(param_grid)
# xg_model.train_models(xg_grouped)
# xg_model.predict_and_plot(xg_grouped)


##### **LSTM**

Long Short-Term Memory (LSTM) is a type of recurrent neural network (RNN) architecture designed to address the vanishing gradient problem in traditional RNNs. LSTMs have a unique memory cell, input gate, forget gate, and output gate, allowing them to capture and remember long-term dependencies in sequential data. They are widely used for tasks involving time series prediction, natural language processing, and other sequential data applications.

In [ ]:

# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

# class CarParkLSTM:
#     def __init__(self, df, facility_name, target_variable = "availability", sequence_length =10, epochs = 50, batch_size = 32):
#         self.df = df[df["facility_name"] == facility_name].copy()
#         self.target_variable = target_variable
#         self.sequence_length = sequence_length
#         self.epochs = epochs
#         self.batch_size = batch_size
#         self.scaler = MinMaxScaler(feature_range=(0, 1))
#         self.model = self.build_model()

#     def build_model(self):
#         model = Sequential()
#         model.add(LSTM(units=50, activation='relu', input_shape=(self.sequence_length, 1)))
#         model.add(LSTM(units=30, activation='relu', input_shape=(self.sequence_length, 1)))
#         model.add(Dense(units=1))
#         model.compile(optimizer='adam', loss='mean_squared_error')
#         return model

#     def prepare_data(self):
#         # Scale the target variable values between 0 and 1
#         self.df['scaled_target'] = self.scaler.fit_transform(self.df[self.target_variable].values.reshape(-1, 1))

#         # Create sequences of data for training
#         data = self.df['scaled_target'].values
#         sequences = []
#         for i in range(len(data) - self.sequence_length):
#             sequences.append(data[i:i + self.sequence_length])
#         sequences = np.array(sequences)

#         # Split data into input (X) and output (y)
#         X = sequences[:, :-1]
#         y = sequences[:, -1]

#         # Reshape for LSTM input shape (samples, time steps, features)
#         X = X.reshape(X.shape[0], X.shape[1], 1)

#         return X, y

#     def train_model(self):
#         X, y = self.prepare_data()
#         self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=0)

#     def predict(self, future_periods=7):
#         last_sequence = self.df['scaled_target'].values[-self.sequence_length:]
#         last_sequence = last_sequence.reshape(1, -1, 1)
#         predictions_scaled = []

#         for _ in range(future_periods):
#             prediction_scaled = self.model.predict(last_sequence)
#             predictions_scaled.append(prediction_scaled[0, 0])
#             last_sequence = np.append(last_sequence[:, 1:, :], [[prediction_scaled[0, 0]]], axis=1)

#         predictions_scaled = np.array(predictions_scaled)
#         predictions = self.scaler.inverse_transform(predictions_scaled.reshape(-1, 1))

#         return predictions

#     def provide_forecast(self, start_datetime, forecast_period=7):
#         self.train_model()

#         # Find the last available datetime in the dataset
#         last_datetime = self.df['timestamp'].max()

#         # Generate a sequence of datetimes for forecasting
#         forecast_datetimes = pd.date_range(start=last_datetime + pd.Timedelta(minutes=15), periods=forecast_period*96, freq='15T')

#         # Make predictions for the specified period
#         predictions = self.predict(forecast_period*96)

#         # Create a DataFrame with forecast datetimes and predictions
#         forecast_df = pd.DataFrame({'timestamp': forecast_datetimes, 'predicted_availability': predictions.flatten()})

#         # Filter forecast for the specified start datetime and period
#         forecast_result = forecast_df[forecast_df['timestamp'] >= start_datetime].head(forecast_period)

#         return forecast_result

#     def plot_predictions(self, future_periods=7):
#         self.train_model()
#         predictions = self.predict(future_periods)

#         plt.figure(figsize=(12, 6))
#         plt.plot(self.df['timestamp'], self.df[self.target_variable], label='Actual Availability', marker='o')
#         plt.plot(pd.date_range(start=self.df['timestamp'].iloc[-1], periods=future_periods*96 + 1, freq='15T')[1:],
#                 predictions, label='Predicted Availability', marker='o')
#         plt.title(f"Car Park Facility {self.df['facility_name'].iloc[0]} Availability Prediction")
#         plt.xlabel('Date')
#         plt.ylabel('Availability')
#         plt.legend()
#         plt.show()

# # Example usage:
# # Assuming your DataFrame is named 'car_park_data' with columns 'timestamp', 'facility_name', and 'availability'
# for facility_name in xg_grouped['facility_name'].unique():
#     lstm_model = CarParkLSTM(df= xg_grouped, facility_name=facility_name, target_variable='y')
#     lstm_model.plot_predictions(future_periods=7)

# # For providing forecasts for a specific period
# forecast_start_datetime = pd.to_datetime('2024-02-11 12:00:00') # Enter the desired start datetime
# forecast_period = 7 # Enter the desired forecast period (in days)
# forecast_result = lstm_model.provide_forecast(start)
# print(forecast_result)

### **6. Deployment**

### **7. Conclusion**

## Conclusion